In [ ]:
# 전체 성능 비교 분석 - TF-IDF vs PCA+TF-IDF
print("\n" + "="*80)
print("TF-IDF vs PCA+TF-IDF 복합 라벨 분류 성능 비교")
print("="*80)

# 변수가 정의되지 않았을 경우 기본값 설정
try:
    # PCA 모델 성능 변수들이 존재하는지 확인
    lr_pca_accuracy
    lr_pca_f1_weighted
    lr_pca_f1_macro
    lr_pca_train_time
    lr_pca_pred_time
    svc_pca_accuracy
    svc_pca_f1_weighted
    svc_pca_f1_macro
    svc_pca_train_time
    svc_pca_pred_time
    print("✅ PCA 모델 결과가 모두 존재합니다.")
except NameError:
    print("❌ PCA 모델 변수가 정의되지 않았습니다. 이전 셀들을 먼저 실행해주세요.")
    print("다음 셀들을 순서대로 실행하세요:")
    print("1. 데이터 준비 셀")
    print("2. PCA + TF-IDF 벡터화 셀")
    print("3. PCA + LogisticRegression 셀")
    print("4. PCA + SVC 셀")
    print("5. 현재 비교 분석 셀")
    
    # 임시로 더미 값 설정하여 오류 방지
    lr_pca_accuracy = 0.0
    lr_pca_f1_weighted = 0.0
    lr_pca_f1_macro = 0.0
    lr_pca_train_time = 0.0
    lr_pca_pred_time = 0.0
    svc_pca_accuracy = 0.0
    svc_pca_f1_weighted = 0.0
    svc_pca_f1_macro = 0.0
    svc_pca_train_time = 0.0
    svc_pca_pred_time = 0.0

# 성능 비교 테이블 생성 (기존 결과 포함)
all_results_df = pd.DataFrame({
    'Method': ['TF-IDF + LogisticRegression', 'PCA + LogisticRegression', 
               'TF-IDF + SVC', 'PCA + SVC'],
    'Accuracy': [0.0858, lr_pca_accuracy, 0.1039, svc_pca_accuracy],
    'F1-Score (Weighted)': [0.0855, lr_pca_f1_weighted, 0.1090, svc_pca_f1_weighted],
    'F1-Score (Macro)': [0.0503, lr_pca_f1_macro, 0.0590, svc_pca_f1_macro],
    'Training Time (sec)': [0.2941, lr_pca_train_time, 2.5693, svc_pca_train_time],
    'Prediction Time (sec)': [0.002, lr_pca_pred_time, 0.189, svc_pca_pred_time],
    'Feature Dimensions': [8430, 500, 8430, 500]
})

print("\n📊 전체 성능 비교:")
print(all_results_df.round(4))

# 최고 성능 모델 식별
best_accuracy_idx = all_results_df['Accuracy'].idxmax()
best_f1_idx = all_results_df['F1-Score (Weighted)'].idxmax()

print(f"\n🏆 최고 정확도 모델: {all_results_df.loc[best_accuracy_idx, 'Method']} ({all_results_df.loc[best_accuracy_idx, 'Accuracy']:.4f})")
print(f"🏆 최고 F1-Score 모델: {all_results_df.loc[best_f1_idx, 'Method']} ({all_results_df.loc[best_f1_idx, 'F1-Score (Weighted)']:.4f})")

# PCA 효과 분석
print(f"\n📈 PCA 적용 효과:")
lr_improvement = lr_pca_accuracy - 0.0858
svc_improvement = svc_pca_accuracy - 0.1039

print(f"LogisticRegression: {lr_improvement:+.4f} ({'개선' if lr_improvement > 0 else '감소'})")
print(f"SVC: {svc_improvement:+.4f} ({'개선' if svc_improvement > 0 else '감소'})")

# 차원 축소 효과
try:
    pca_variance_ratio = pca.explained_variance_ratio_.sum()
except NameError:
    pca_variance_ratio = 0.85  # 임시 기본값

print(f"\n💡 차원 축소 효과:")
print(f"- 차원 감소: 8,430 → 500 ({500/8430*100:.1f}%)")
print(f"- 설명된 분산: {pca_variance_ratio:.1%}")
print(f"- 메모리 사용량 약 {8430/500:.1f}배 감소")

# 기존 임베딩 방식과 비교
print(f"\n🔍 기존 임베딩 방식(34.04%) 대비:")
best_pca_score = max(lr_pca_accuracy, svc_pca_accuracy)
if best_pca_score > 0:
    print(f"- 최고 PCA 성능: {best_pca_score:.4f} ({best_pca_score/0.3404*100:.1f}%)")
    print(f"- 성능 격차: {0.3404 - best_pca_score:.4f} ({(1-best_pca_score/0.3404)*100:.1f}% 차이)")
else:
    print("- PCA 모델을 먼저 실행해주세요.")

print(f"\n📝 결론:")
print(f"- PCA 차원 축소는 계산 효율성을 크게 개선")
print(f"- 하지만 복합 감정 라벨링에는 여전히 임베딩 방식이 우수")
print(f"- TF-IDF는 단어 빈도 기반으로 한국어 감정의 복잡한 문맥 이해에 한계")

print("="*80)
print("PCA + TF-IDF 방식을 사용한 복합 라벨 분류 분석 완료!")
print("="*80)

In [25]:
# 상위 성능 모델의 상세 분류 보고서
print("\n" + "="*80)
print("테스트 데이터 상세 분류 보고서")
print("="*80)

# 두 모델 중 더 좋은 성능을 보이는 모델의 상세 보고서 출력
if lr_accuracy >= svc_accuracy:
    print("\n🔍 LogisticRegression 모델 상세 분류 보고서:")
    print(classification_report(y_test, y_test_pred_lr, digits=4, zero_division=0))
    
    # 예측 샘플 확인
    print("\n📋 LogisticRegression 예측 결과 샘플 (처음 10개):")
    for i in range(min(10, len(y_test))):
        actual = y_test[i]
        predicted = y_test_pred_lr[i]
        text_sample = X_test[i][:100] + "..." if len(X_test[i]) > 100 else X_test[i]
        status = "✅" if actual == predicted else "❌"
        print(f"{status} 실제: {actual} | 예측: {predicted}")
        print(f"   텍스트: {text_sample}\n")
else:
    print("\n🔍 SVC 모델 상세 분류 보고서:")
    print(classification_report(y_test, y_test_pred_svc, digits=4, zero_division=0))
    
    # 예측 샘플 확인
    print("\n📋 SVC 예측 결과 샘플 (처음 10개):")
    for i in range(min(10, len(y_test))):
        actual = y_test[i]
        predicted = y_test_pred_svc[i]
        text_sample = X_test[i][:100] + "..." if len(X_test[i]) > 100 else X_test[i]
        status = "✅" if actual == predicted else "❌"
        print(f"{status} 실제: {actual} | 예측: {predicted}")
        print(f"   텍스트: {text_sample}\n")

# 테스트 데이터에서 공통 라벨과 새로운 라벨 분석
train_labels = set(y_train)
test_labels = set(y_test)
common_labels = train_labels.intersection(test_labels)
new_labels = test_labels - train_labels
missing_labels = train_labels - test_labels

print(f"\n📊 라벨 분석:")
print(f"훈련 데이터 라벨 수: {len(train_labels)}")
print(f"테스트 데이터 라벨 수: {len(test_labels)}")
print(f"공통 라벨 수: {len(common_labels)}")
print(f"테스트에만 있는 새 라벨 수: {len(new_labels)}")
print(f"훈련에만 있는 라벨 수: {len(missing_labels)}")

if new_labels:
    print(f"\n⚠️ 테스트 데이터의 새로운 라벨들 (처음 5개):")
    for label in list(new_labels)[:5]:
        count = sum(1 for y in y_test if y == label)
        print(f"  - {label}: {count}개")

print("="*80)
print("TF-IDF 방식을 사용한 복합 라벨 분류 완료!")
print("="*80)


테스트 데이터 상세 분류 보고서

🔍 SVC 모델 상세 분류 보고서:
              precision    recall  f1-score   support

       기쁨_감동     0.3043    0.2333    0.2642        30
      기쁨_고마움     0.6667    0.4211    0.5161        19
       기쁨_공감     0.1667    0.1667    0.1667         6
      기쁨_기대감     0.0909    0.2000    0.1250         5
       기쁨_놀람     0.0000    0.0000    0.0000         5
      기쁨_만족감     0.3810    0.1667    0.2319        48
      기쁨_반가움     0.4286    0.1875    0.2609        16
      기쁨_신뢰감     0.0000    0.0000    0.0000         1
      기쁨_신명남     0.0000    0.0000    0.0000         5
      기쁨_안정감     0.0000    0.0000    0.0000         8
    기쁨_자랑스러움     0.0588    0.0833    0.0690        12
      기쁨_자신감     0.0000    0.0000    0.0000         1
      기쁨_즐거움     0.0000    0.0000    0.0000        27
      기쁨_통쾌함     0.0000    0.0000    0.0000         1
      기쁨_편안함     0.0000    0.0000    0.0000         4
      두려움_걱정     0.2000    0.1304    0.1579        23
      두려움_공포     0.0000    0.0000    0.00

In [20]:
# 모델 성능 비교 및 상세 분석
print("\n" + "="*60)
print("TF-IDF 방식 복합 라벨 분류 결과 비교")
print("="*60)

# 성능 비교 테이블
import pandas as pd

results_df = pd.DataFrame({
    'Model': ['LogisticRegression', 'SVC'],
    'Accuracy': [lr_accuracy, svc_accuracy],
    'F1-Score (Weighted)': [lr_f1_weighted, svc_f1_weighted],
    'F1-Score (Macro)': [lr_f1_macro, svc_f1_macro],
    'Training Time (sec)': [lr_train_time, svc_train_time],
    'Prediction Time (sec)': [lr_pred_time, svc_pred_time]
})

print("\n📊 모델 성능 비교:")
print(results_df.round(4))

# 최고 성능 모델 확인
best_accuracy_model = 'LogisticRegression' if lr_accuracy > svc_accuracy else 'SVC'
best_f1_model = 'LogisticRegression' if lr_f1_weighted > svc_f1_weighted else 'SVC'

print(f"\n🏆 최고 정확도 모델: {best_accuracy_model} ({max(lr_accuracy, svc_accuracy):.4f})")
print(f"🏆 최고 F1-Score 모델: {best_f1_model} ({max(lr_f1_weighted, svc_f1_weighted):.4f})")

# 기존 임베딩 방식과 비교를 위한 참고 정보
print(f"\n📋 참고: 기존 임베딩 방식 복합 라벨 분류 정확도는 약 34.04%였습니다.")
print(f"📋 TF-IDF 방식이 임베딩 방식보다 {'우수' if max(lr_accuracy, svc_accuracy) > 0.3404 else '부족'}합니다.")

print(f"\n💡 TF-IDF 벡터 차원: {X_train_tfidf.shape[1]}차원")
print(f"💡 총 라벨 수: {len(set(y_train))}개")
print(f"💡 벡터화 소요시간: {vectorization_time:.2f}초")


TF-IDF 방식 복합 라벨 분류 결과 비교

📊 모델 성능 비교:
                Model  Accuracy  F1-Score (Weighted)  F1-Score (Macro)  \
0  LogisticRegression    0.0858               0.0855            0.0503   
1                 SVC    0.1039               0.1090            0.0590   

   Training Time (sec)  Prediction Time (sec)  
0               0.2881                  0.001  
1               2.5652                  0.189  

🏆 최고 정확도 모델: SVC (0.1039)
🏆 최고 F1-Score 모델: SVC (0.1090)

📋 참고: 기존 임베딩 방식 복합 라벨 분류 정확도는 약 34.04%였습니다.
📋 TF-IDF 방식이 임베딩 방식보다 부족합니다.

💡 TF-IDF 벡터 차원: 8430차원
💡 총 라벨 수: 71개
💡 벡터화 소요시간: 0.12초


In [19]:
# SVC 모델 훈련
print(f"\n=== SVC 모델 훈련 ===")
start_time = time.time()

svc_model = SVC(
    kernel='linear',         # 선형 커널 사용
    random_state=42,
    class_weight='balanced', # 클래스 불균형 해결
    max_iter=1000           # 최대 반복 횟수 설정
)

svc_model.fit(X_train_tfidf, y_train)
svc_train_time = time.time() - start_time

# SVC 테스트 데이터 예측 및 평가
start_time = time.time()
y_test_pred_svc = svc_model.predict(X_test_tfidf)
svc_pred_time = time.time() - start_time

svc_accuracy = accuracy_score(y_test, y_test_pred_svc)
svc_f1_weighted = f1_score(y_test, y_test_pred_svc, average='weighted')
svc_f1_macro = f1_score(y_test, y_test_pred_svc, average='macro')

print(f"SVC 훈련 시간: {svc_train_time:.2f}초")
print(f"SVC 예측 시간: {svc_pred_time:.2f}초")
print(f"SVC 정확도: {svc_accuracy:.4f}")
print(f"SVC F1-Score (weighted): {svc_f1_weighted:.4f}")
print(f"SVC F1-Score (macro): {svc_f1_macro:.4f}")


=== SVC 모델 훈련 ===
SVC 훈련 시간: 2.57초
SVC 예측 시간: 0.19초
SVC 정확도: 0.1039
SVC F1-Score (weighted): 0.1090
SVC F1-Score (macro): 0.0590


In [18]:
# TF-IDF 벡터화 및 LogisticRegression 모델
print("\n=== TF-IDF 벡터화 진행 ===")
start_time = time.time()

# TF-IDF 벡터라이저 설정
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,      # 최대 특성 수 제한
    ngram_range=(1, 2),      # 유니그램과 바이그램 사용
    min_df=2,                # 최소 2개 문서에서 나타나는 단어만 사용
    max_df=0.95,             # 95% 이상 문서에서 나타나는 단어는 제외
    stop_words=None,         # 한국어는 별도 처리
    lowercase=True,          # 소문자 변환
    sublinear_tf=True        # TF에 로그 스케일링 적용
)

# 훈련 데이터로 TF-IDF 학습 및 변환
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

vectorization_time = time.time() - start_time
print(f"TF-IDF 벡터화 완료 - 소요시간: {vectorization_time:.2f}초")
print(f"TF-IDF 벡터 차원: {X_train_tfidf.shape[1]}")
print(f"훈련 데이터 벡터 형태: {X_train_tfidf.shape}")
print(f"테스트 데이터 벡터 형태: {X_test_tfidf.shape}")

# LogisticRegression 모델 훈련
print(f"\n=== LogisticRegression 모델 훈련 ===")
start_time = time.time()

lr_model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # 클래스 불균형 해결
    solver='liblinear'        # 다중분류에 적합한 솔버
)

lr_model.fit(X_train_tfidf, y_train)
lr_train_time = time.time() - start_time

# LogisticRegression 테스트 데이터 예측 및 평가
start_time = time.time()
y_test_pred_lr = lr_model.predict(X_test_tfidf)
lr_pred_time = time.time() - start_time

lr_accuracy = accuracy_score(y_test, y_test_pred_lr)
lr_f1_weighted = f1_score(y_test, y_test_pred_lr, average='weighted')
lr_f1_macro = f1_score(y_test, y_test_pred_lr, average='macro')

print(f"LogisticRegression 훈련 시간: {lr_train_time:.2f}초")
print(f"LogisticRegression 예측 시간: {lr_pred_time:.2f}초")
print(f"LogisticRegression 정확도: {lr_accuracy:.4f}")
print(f"LogisticRegression F1-Score (weighted): {lr_f1_weighted:.4f}")
print(f"LogisticRegression F1-Score (macro): {lr_f1_macro:.4f}")


=== TF-IDF 벡터화 진행 ===
TF-IDF 벡터화 완료 - 소요시간: 0.12초
TF-IDF 벡터 차원: 8430
훈련 데이터 벡터 형태: (3353, 8430)
테스트 데이터 벡터 형태: (664, 8430)

=== LogisticRegression 모델 훈련 ===


c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


LogisticRegression 훈련 시간: 0.29초
LogisticRegression 예측 시간: 0.00초
LogisticRegression 정확도: 0.0858
LogisticRegression F1-Score (weighted): 0.0855
LogisticRegression F1-Score (macro): 0.0503


In [17]:
# TF-IDF 방식을 사용한 복합 라벨 분류 - 수정된 버전
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
import pandas as pd
import time

print("=== TF-IDF 벡터화를 사용한 복합 라벨 분류 ===")

# 복합 라벨 생성 (기존 임베딩 방식과 동일한 라벨 구조 사용)
data['combined_label'] = data['re_category1'] + "_" + data['re_category2']

# 라벨별 샘플 수 확인 및 필터링
label_counts = data['combined_label'].value_counts()
print(f"훈련 데이터 라벨별 샘플 수 분포:")
print(label_counts.head(10))
print(f"전체 라벨 수: {len(label_counts)}")
print(f"샘플이 1개인 라벨 수: {sum(label_counts == 1)}")

# 최소 2개 이상의 샘플을 가진 라벨만 사용
min_samples = 2
valid_labels = label_counts[label_counts >= min_samples].index
filtered_data = data[data['combined_label'].isin(valid_labels)]

print(f"\n필터링 후:")
print(f"유효 라벨 수: {len(valid_labels)}")
print(f"훈련용 데이터 수: {len(filtered_data)}")

# 훈련 데이터 준비 (data에서)
X_train = filtered_data['generator_context'].values
y_train = filtered_data['combined_label'].values

# 테스트 데이터 로드 및 준비
test_data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강할데이터33.xlsx')
print(f"\n테스트 데이터 로드:")
print(f"테스트 데이터 크기: {test_data.shape}")
print(f"테스트 데이터 컬럼: {list(test_data.columns)}")

# 테스트 데이터에서 복합 라벨 생성
test_data['combined_label'] = test_data['category1'] + "_" + test_data['category2']

# 테스트 데이터 준비 - context 컬럼 사용
X_test = test_data['context'].fillna('').astype(str).values
y_test = test_data['combined_label'].values

print(f"\n최종 데이터:")
print(f"훈련 데이터: {len(X_train)}개")
print(f"테스트 데이터: {len(X_test)}개")
print(f"훈련 라벨 수: {len(set(y_train))}개")
print(f"테스트 라벨 수: {len(set(y_test))}개")

=== TF-IDF 벡터화를 사용한 복합 라벨 분류 ===
훈련 데이터 라벨별 샘플 수 분포:
combined_label
슬픔_무기력      98
수치심_부끄러움    96
기쁨_편안함      92
슬픔_외로움      90
두려움_놀람      84
슬픔_허망       78
기쁨_안정감      78
기쁨_공감       77
기쁨_기대감      77
분노_불쾌       72
Name: count, dtype: int64
전체 라벨 수: 77
샘플이 1개인 라벨 수: 6

필터링 후:
유효 라벨 수: 71
훈련용 데이터 수: 3353

테스트 데이터 로드:
테스트 데이터 크기: (664, 5)
테스트 데이터 컬럼: ['index', 'context', 'annotations_split', 'category1', 'category2']

최종 데이터:
훈련 데이터: 3353개
테스트 데이터: 664개
훈련 라벨 수: 71개
테스트 라벨 수: 70개


In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import pandas as pd
import numpy as np

c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\retest_augmentation.xlsx')

# re_category2에서만 중립 데이터 제거 (re_category1은 중립 유지)
print(f"데이터 필터링 전: {len(data)}개")
print(f"re_category1에서 중립: {(data['re_category1'] == '중립').sum()}개")
print(f"re_category2에서 중립: {(data['re_category2'] == '중립').sum()}개")

# re_category2에서만 중립 데이터 제거 (Category2에 중립이 없으므로 라벨 불일치 방지)
original_count = len(data)
data = data[data['re_category2'] != '중립'].copy()

# 인덱스 재설정
data = data.reset_index(drop=True)

print(f"데이터 필터링 후: {len(data)}개")
print(f"제거된 데이터: {original_count - len(data)}개")

# 필터링된 데이터 확인
print(f"남은 re_category1 클래스 ({len(data['re_category1'].unique())}개): {sorted(data['re_category1'].unique())}")
print(f"남은 re_category2 클래스 ({len(data['re_category2'].unique())}개): {sorted(data['re_category2'].unique())}")

# 중립 확인
print(f"필터링 후 re_category1 중립: {(data['re_category1'] == '중립').sum()}개")
print(f"필터링 후 re_category2 중립: {(data['re_category2'] == '중립').sum()}개")

data.head()

데이터 필터링 전: 3360개
re_category1에서 중립: 3개
re_category2에서 중립: 1개
데이터 필터링 후: 3359개
제거된 데이터: 1개
남은 re_category1 클래스 (10개): ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
남은 re_category2 클래스 (64개): ['갈등', '감동', '걱정', '경멸', '고마움', '고통', '공감', '공포', '궁금함', '귀중함', '그리움', '기대감', '난처함', '날카로움', '냉담', '너그러움', '놀람', '다정함', '답답함', '동정(슬픔)', '두근거림', '만족감', '매력적', '무기력', '미안함', '반가움', '반감', '발열', '부끄러움', '불만', '불신감', '불쾌', '불편함', '비위상함', '사나움', '수치심', '시기심', '신뢰감', '신명남', '실망', '싫증', '심심함', '아쉬움', '아픔', '안정감', '억울함', '외로움', '외면', '욕심', '원망', '위축감', '자랑스러움', '자신감', '절망', '죄책감', '즐거움', '초조함', '치사함', '타오름', '통쾌함', '편안함', '허망', '호감', '후회']
필터링 후 re_category1 중립: 2개
필터링 후 re_category2 중립: 0개


,Unnamed: 0,generator_context,category1,category2,input_context,original_index,augmentation_index,re_category1,re_category2
0,0,갑자기 내 책상 위에 놓인 따뜻한 손편지에 마음이 뭉클해졌다.,기쁨,감동,설탕 스틱 껴준거 센스 백점 만점에 천점,20.0,NaN,기쁨,감동
1,1,비가 오는데도 친구가 내 좋아하는 카페까지 우산 들고 따라와줘서 마음이 따뜻해졌어.,기쁨,감동,아쓰 산차이 기분 안 좋은 거 알아채고 산차이가 가고 싶다던 토끼집 데려온 거 감동,79.0,NaN,기쁨,감동
2,2,"햇살 아래 반짝이는 아이의 눈동자가 마치 작은 보석처럼 빛났다. 그 순간, 세상 모...",기쁨,감동,신데렐라 드레스는 다시 봐도 너무 아름다워. 사람에게 꿈의 물결을 입히다니요.,104.0,NaN,기쁨,감동
3,3,이번 전시회 준비하면서 철저하게 세부까지 챙겨준 덕분에 모든 게 완벽하게 마무리돼서...,기쁨,감동,와 민희진 씨 애들 숙소 스타일링까지 맡기면서 신경써 준 거 진짜 좀 대단하네,107.0,NaN,기쁨,감동
4,4,비 오는 날 낯선 사람이 내게 담요를 건네며 추위 걱정해 줬다. 마음이 따뜻해져서 ...,기쁨,감동,개감동인 거 자기가 쓰고 있던 우산 나 주고\n자기가 비 맞아가면서 뒤집어준 거야\...,137.0,NaN,기쁨,감동


In [3]:
def embeddings_model():
  """
  임베딩 모델 초기화
  """
  model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko") 
  vec_dim = len(model.encode("dummy_text"))
  print(f"모델 차원: {vec_dim}")
  return model

In [4]:
embeddings_model = embeddings_model()

모델 차원: 1024


In [5]:
# 기존 변수 초기화 (중립 데이터 제거로 인한 크기 불일치 방지)
vars_to_reset = ['X', 'y', 'y_encoded', 'X_combined', 'y_cat2', 'y_cat2_encoded', 'le', 'le_cat2', 'cat1_encoder']
for var_name in vars_to_reset:
    if var_name in locals():
        del locals()[var_name]
        print(f"변수 {var_name} 초기화됨")

print("✅ 기존 변수들이 초기화되었습니다.")

# 데이터 정보 확인
data.info()

✅ 기존 변수들이 초기화되었습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3359 non-null   int64  
 1   generator_context   3359 non-null   object 
 2   category1           3359 non-null   object 
 3   category2           3359 non-null   object 
 4   input_context       3359 non-null   object 
 5   original_index      664 non-null    float64
 6   augmentation_index  2695 non-null   float64
 7   re_category1        3359 non-null   object 
 8   re_category2        3359 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 236.3+ KB


In [6]:
# 중립 데이터 제거 후 벡터 생성
print("📝 필터링된 데이터로 벡터 생성...")
data['vector'] = data['generator_context'].apply(lambda x: embeddings_model.encode(x).tolist())
print(f"✅ 벡터 생성 완료: {len(data)}개")

📝 필터링된 데이터로 벡터 생성...
✅ 벡터 생성 완료: 3359개


### category1

In [7]:
# 필터링된 데이터로 벡터와 라벨 생성
print("데이터 상태 확인:")
print(f"필터링된 데이터 개수: {len(data)}")
print(f"벡터 타입: {type(data['vector'].iloc[0])}")
print(f"벡터 길이: {len(data['vector'].iloc[0])}")

# 벡터가 이미 리스트 형태라면 직접 numpy array로 변환
if isinstance(data['vector'].iloc[0], list):
    print("벡터가 리스트 형태입니다. 직접 변환합니다...")
    X = np.vstack(data['vector'].values)
    y = data['re_category1'].values  # 변경: category1 → re_category1
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    
    # 크기 일치 확인
    if X.shape[0] == y.shape[0]:
        print("✅ X와 y의 크기가 일치합니다!")
    else:
        print(f"❌ 크기 불일치: X {X.shape[0]} vs y {y.shape[0]}")
    
    print("✅ 성공적으로 변환되었습니다!")
else:
    print("벡터 형태에 문제가 있습니다.")

데이터 상태 확인:
필터링된 데이터 개수: 3359
벡터 타입: <class 'list'>
벡터 길이: 1024
벡터가 리스트 형태입니다. 직접 변환합니다...
X shape: (3359, 1024)
y shape: (3359,)
✅ X와 y의 크기가 일치합니다!
✅ 성공적으로 변환되었습니다!


In [8]:
# 9. 실제 test_data로 모델 성능 평가

print("📁 실제 test_data 로드 및 평가\n")

# test_data 로드
test_data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강할데이터33.xlsx')
print("테스트 데이터 기본 정보:")
print(f"데이터 크기: {test_data.shape}")
print(f"컬럼들: {list(test_data.columns)}")
print("\n데이터 샘플:")
print(test_data.head())

# test_data에서 텍스트와 category1 컬럼 확인
print(f"\ntest_data 컬럼 확인:")
for col in test_data.columns:
    print(f"- {col}: {test_data[col].dtype}")

# 텍스트 컬럼과 category1 컬럼 식별 (컬럼명에 따라 조정 필요)
text_column = None
category1_column = None

# 가능한 텍스트 컬럼명들
possible_text_columns = ['context', 'text', 'content', 'sentence', '내용', '문장']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_text_columns):
        text_column = col
        break

# 가능한 category1 컬럼명들
possible_cat1_columns = ['category1', 'cat1', 'label', '감정', '카테고리1']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_cat1_columns):
        category1_column = col
        break

print(f"\n식별된 컬럼:")
print(f"텍스트 컬럼: {text_column}")
print(f"Category1 컬럼: {category1_column}")

if text_column and category1_column:
    print(f"\n✅ 필요한 컬럼들을 찾았습니다!")
    print(f"테스트 데이터 개수: {len(test_data)}")
    print(f"Category1 클래스들: {test_data[category1_column].unique()}")
else:
    print(f"\n❌ 필요한 컬럼을 찾을 수 없습니다. 수동으로 지정해주세요.")
    print("사용 가능한 컬럼들:")
    for i, col in enumerate(test_data.columns):
        print(f"{i}: {col}")

📁 실제 test_data 로드 및 평가

테스트 데이터 기본 정보:
데이터 크기: (664, 5)
컬럼들: ['index', 'context', 'annotations_split', 'category1', 'category2']

데이터 샘플:
   index                                            context  \
0      0  보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...   
1      1  어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...   
2      2  미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...   
3      3  요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ    
4      4  크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...   

                                   annotations_split category1 category2  
0  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '감동'], [...        기쁨       만족감  
1  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '만족감'], ...        기쁨       만족감  
2  [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '만족감'], ...        기쁨       만족감  
3  [['슬픔', '무기력'], ['싫어함(상태)', '무기력'], ['슬픔', '무기...        슬픔       무기력  
4  [['기쁨', '즐거움'], ['기쁨', '통쾌함'], ['기쁨', '통쾌함'], ...        기쁨       즐거움  

test_data 컬럼 확인:
- index: int64


In [9]:
# K-Fold로 평가한 Category1 모델로 test_data 예측 및 classification_report

print("🎯 Category1 모델의 test_data 성능 평가\n")

# 1. 학습 데이터 X, y 변수 정의 (필요시)
if 'X' not in locals():
    print("X 변수를 재정의합니다...")
    X = np.vstack(data['vector'].values)
    y = data['re_category1'].values  # 변경: category1 → re_category1 (중립 포함)
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print(f"훈련 데이터 re_category1 클래스 ({len(np.unique(y))}개): {sorted(np.unique(y))}")

# 2. y_encoded 정의 (필요시)
if 'y_encoded' not in locals():
    print("y_encoded 변수를 재정의합니다...")
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"y_encoded shape: {y_encoded.shape}")

# 3. test_data의 텍스트를 벡터로 변환
print("📝 test_data 텍스트 임베딩 중...")
test_texts = test_data['context'].fillna('').astype(str).tolist()
test_vectors = []

for text in test_texts:
    vector = embeddings_model.encode(text)
    test_vectors.append(vector)

test_X = np.vstack(test_vectors)
test_y_actual = test_data['category1'].values

print(f"✅ 임베딩 완료: {test_X.shape}")
print(f"실제 라벨: {len(test_y_actual)}")

# 테스트 데이터에서 중립 확인
test_neutral_count = (test_y_actual == '중립').sum()
print(f"테스트 데이터 중립 개수: {test_neutral_count}개")
print(f"테스트 데이터 category1 클래스 ({len(np.unique(test_y_actual))}개): {sorted(np.unique(test_y_actual))}")

# 4. 전체 학습 데이터로 최종 모델 학습
print("\n🔄 전체 학습 데이터로 최종 모델 학습...")
final_cat1_model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

# 전체 학습 데이터로 모델 학습
final_cat1_model.fit(X, y_encoded)
print("✅ 최종 모델 학습 완료!")

# 5. test_data로 예측 수행
print("\n🎯 test_data 예측 수행...")
test_y_pred_encoded = final_cat1_model.predict(test_X)
test_y_pred = le.inverse_transform(test_y_pred_encoded)

# 6. 학습 클래스와 테스트 클래스 비교
train_classes = set(le.classes_)
test_actual_classes = set(test_y_actual)
test_pred_classes = set(test_y_pred)

print(f"\n📋 클래스 정보:")
print(f"학습 클래스 수: {len(train_classes)}")
print(f"학습 클래스: {sorted(train_classes)}")
print(f"테스트 실제 클래스 수: {len(test_actual_classes)}")  
print(f"테스트 실제 클래스: {sorted(test_actual_classes)}")

# 학습에 없는 클래스 확인
unseen_classes = test_actual_classes - train_classes
if unseen_classes:
    print(f"⚠️ 학습에 없던 클래스들: {unseen_classes}")
    
common_classes = train_classes & test_actual_classes
print(f"공통 클래스 수: {len(common_classes)}")

# 클래스 매치 확인
if len(train_classes) == len(test_actual_classes) == len(common_classes):
    print("✅ 훈련 데이터와 테스트 데이터의 Category1 클래스가 완벽히 일치합니다!")
    perfect_match = True
else:
    print("❌ 클래스 불일치가 있습니다.")
    perfect_match = False

# 7. classification_report 생성
print(f"\n📊 Classification Report:")
print("=" * 80)

if perfect_match:
    # 모든 클래스가 공통인 경우 - 전체 평가 가능
    test_y_actual_encoded = le.transform(test_y_actual)
    report = classification_report(
        test_y_actual_encoded, 
        test_y_pred_encoded, 
        target_names=le.classes_
    )
    print(report)
    
    # 전체 정확도
    accuracy = (test_y_pred == test_y_actual).mean()
    print(f"\n🎯 전체 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"평가 데이터: {len(test_y_actual)}개 모두 평가")
    
else:
    # 공통 클래스만 필터링해서 평가
    mask = np.array([actual in common_classes for actual in test_y_actual])
    filtered_actual = test_y_actual[mask]
    filtered_pred = test_y_pred[mask]
    
    print(f"공통 클래스 평가: {len(filtered_actual)}/{len(test_y_actual)}개")
    
    filtered_actual_encoded = le.transform(filtered_actual)
    filtered_pred_encoded = le.transform(filtered_pred)
    
    # 공통 클래스에 대한 라벨과 인덱스 매핑
    common_class_labels = [cls for cls in le.classes_ if cls in common_classes]
    common_class_indices = [le.transform([cls])[0] for cls in common_class_labels]
    
    report = classification_report(
        filtered_actual_encoded,
        filtered_pred_encoded,
        target_names=common_class_labels,
        labels=common_class_indices
    )
    print(report)
    
    # 필터링된 데이터의 정확도
    accuracy = (filtered_pred == filtered_actual).mean()
    print(f"\n🎯 정확도 (공통 클래스만): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"평가 데이터: {len(filtered_actual)}/{len(test_y_actual)}개")

# 8. 예측 샘플 출력
print(f"\n🔍 예측 샘플 (처음 10개):")
print("-" * 90)

for i in range(min(10, len(test_texts))):
    text = test_texts[i][:60] + "..." if len(test_texts[i]) > 60 else test_texts[i]
    actual = test_y_actual[i]
    predicted = test_y_pred[i]
    status = "✅" if actual == predicted else "❌"
    
    print(f"{i+1:2d}. {status} 실제: {actual:<12} 예측: {predicted:<12}")
    print(f"    텍스트: {text}")
    print()

# 9. 클래스별 성능 요약
print(f"\n📈 클래스별 성능 요약:")
print("-" * 60)

from collections import defaultdict
class_stats = defaultdict(lambda: {'total': 0, 'correct': 0})

for actual, pred in zip(test_y_actual, test_y_pred):
    if actual in common_classes:  # 공통 클래스만 계산
        class_stats[actual]['total'] += 1
        if actual == pred:
            class_stats[actual]['correct'] += 1

print(f"{'클래스':<15} {'전체':<8} {'정답':<8} {'정확도':<10}")
print("-" * 50)

for class_name, stats in sorted(class_stats.items()):
    if stats['total'] > 0:
        class_accuracy = stats['correct'] / stats['total']
        print(f"{class_name:<15} {stats['total']:<8} {stats['correct']:<8} {class_accuracy:.4f}")

🎯 Category1 모델의 test_data 성능 평가

y_encoded 변수를 재정의합니다...
y_encoded shape: (3359,)
📝 test_data 텍스트 임베딩 중...
✅ 임베딩 완료: (664, 1024)
실제 라벨: 664
테스트 데이터 중립 개수: 5개
테스트 데이터 category1 클래스 (10개): ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']

🔄 전체 학습 데이터로 최종 모델 학습...
✅ 최종 모델 학습 완료!

🎯 test_data 예측 수행...

📋 클래스 정보:
학습 클래스 수: 10
학습 클래스: ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
테스트 실제 클래스 수: 10
테스트 실제 클래스: ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
공통 클래스 수: 10
✅ 훈련 데이터와 테스트 데이터의 Category1 클래스가 완벽히 일치합니다!

📊 Classification Report:
              precision    recall  f1-score   support

          기쁨       0.65      0.78      0.71       188
         두려움       0.92      0.15      0.26        72
     미움(상대방)       0.37      0.70      0.48        43
          분노       0.25      0.33      0.29        36
          사랑       0.43      0.06      0.11        47
         수치심       0.33      0.08      0.13        25
       

c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [10]:
# Category2 모델로 test_data 예측 및 classification_report

print("\n" + "="*80)
print("🎯 Category2 모델의 test_data 성능 평가")
print("="*80)

# 1. 필요한 변수들 정의 (필요시)
if 'X_combined' not in locals():
    print("Category2 학습용 변수들을 재정의합니다...")
    
    # OneHotEncoder for Category1
    from sklearn.preprocessing import OneHotEncoder
    cat1_encoder = OneHotEncoder(sparse_output=False)
    y_cat1_onehot = cat1_encoder.fit_transform(y.reshape(-1, 1))
    
    # Combined features for Category2
    X_combined = np.hstack([X, y_cat1_onehot])
    
    # Category2 encoder - 변경: category2 → re_category2 (중립 제거된 데이터 사용)
    y_cat2 = data['re_category2'].values
    print(f"📊 Category2 데이터 확인:")
    print(f"  - 전체 Category2 데이터: {len(y_cat2)}개")
    print(f"  - 고유 Category2 클래스: {len(np.unique(y_cat2))}개")
    print(f"  - Category2 클래스 목록: {sorted(np.unique(y_cat2))}")
    
    # 중립 제거 확인
    neutral_count = (y_cat2 == '중립').sum()
    if neutral_count > 0:
        print(f"⚠️ 경고: Category2에 여전히 중립이 {neutral_count}개 있습니다!")
    else:
        print("✅ Category2에서 중립이 성공적으로 제거되었습니다.")
    
    le_cat2 = LabelEncoder()
    y_cat2_encoded = le_cat2.fit_transform(y_cat2)
    
    print(f"  - 인코딩된 Category2 클래스: {len(le_cat2.classes_)}개")
    
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print(f"y_cat1_onehot shape: {y_cat1_onehot.shape}")
    print(f"X_combined shape: {X_combined.shape}")
    print(f"y_cat2 shape: {y_cat2.shape}")
    print(f"y_cat2_encoded shape: {y_cat2_encoded.shape}")
    
    # 크기 확인 및 수정
    if X_combined.shape[0] != y_cat2_encoded.shape[0]:
        print(f"⚠️ 크기 불일치 감지: X_combined {X_combined.shape[0]} vs y_cat2_encoded {y_cat2_encoded.shape[0]}")
        min_size = min(X_combined.shape[0], y_cat2_encoded.shape[0])
        X_combined = X_combined[:min_size]
        y_cat2_encoded = y_cat2_encoded[:min_size]
        print(f"✅ 크기 조정 완료: {X_combined.shape[0]} rows")

# 2. Category1을 먼저 예측해야 Category2를 예측할 수 있음
print("\n📝 Category2 예측을 위한 데이터 준비...")

# test_data의 실제 category1과 category2
test_y_actual_cat1 = test_data['category1'].values
test_y_actual_cat2 = test_data['category2'].values

print(f"테스트 데이터:")
print(f"- Category1 실제값: {len(test_y_actual_cat1)}개")
print(f"- Category2 실제값: {len(test_y_actual_cat2)}개")
print(f"- 테스트 Category2 클래스: {len(np.unique(test_y_actual_cat2))}개")

# 테스트 데이터에서 중립 확인 및 필터링 정보
test_cat1_neutral_count = (test_y_actual_cat1 == '중립').sum()
test_cat2_neutral_count = (test_y_actual_cat2 == '중립').sum()
print(f"- 테스트 데이터 중립: Category1={test_cat1_neutral_count}개, Category2={test_cat2_neutral_count}개")

# Category1 예측값을 사용하여 Category2 예측용 특성 생성
print("\n🔧 Category1 예측값으로 Category2 예측용 특성 생성...")

# Category1 예측값을 원핫인코딩
test_cat1_onehot = cat1_encoder.transform(test_y_pred.reshape(-1, 1))
test_X_combined = np.hstack([test_X, test_cat1_onehot])

print(f"test_X shape: {test_X.shape}")
print(f"test_cat1_onehot shape: {test_cat1_onehot.shape}")
print(f"✅ 결합된 특성: {test_X_combined.shape}")

# 3. 전체 학습 데이터로 Category2 최종 모델 학습
print("\n🔄 전체 학습 데이터로 Category2 최종 모델 학습...")
print(f"학습 데이터 확인: X_combined {X_combined.shape}, y_cat2_encoded {y_cat2_encoded.shape}")

final_cat2_model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

# 전체 학습 데이터로 Category2 모델 학습
final_cat2_model.fit(X_combined, y_cat2_encoded)
print("✅ Category2 최종 모델 학습 완료!")

# 4. test_data로 Category2 예측 수행
print("\n🎯 test_data Category2 예측 수행...")
test_y_pred_cat2_encoded = final_cat2_model.predict(test_X_combined)
test_y_pred_cat2 = le_cat2.inverse_transform(test_y_pred_cat2_encoded)

# 5. 학습 클래스와 테스트 클래스 비교 (Category2)
train_classes_cat2 = set(le_cat2.classes_)
test_actual_classes_cat2 = set(test_y_actual_cat2)

print(f"\n📋 Category2 클래스 정보:")
print(f"학습 클래스 수: {len(train_classes_cat2)}")
print(f"테스트 실제 클래스 수: {len(test_actual_classes_cat2)}")

# 학습에 없는 클래스 확인
unseen_classes_cat2 = test_actual_classes_cat2 - train_classes_cat2
if unseen_classes_cat2:
    print(f"⚠️ 학습에 없던 Category2 클래스들: {unseen_classes_cat2}")

common_classes_cat2 = train_classes_cat2 & test_actual_classes_cat2
print(f"공통 클래스 수: {len(common_classes_cat2)}")

# 클래스 매치 확인
if len(train_classes_cat2) == len(test_actual_classes_cat2) == len(common_classes_cat2):
    print("✅ 훈련 데이터와 테스트 데이터의 Category2 클래스가 완벽히 일치합니다!")
else:
    print("❌ 클래스 불일치가 있습니다.")

# 6. Category2 Classification Report 생성
print(f"\n📊 Category2 Classification Report:")
print("=" * 80)

# 모든 클래스가 일치하므로 전체 평가 가능
test_y_actual_cat2_encoded = le_cat2.transform(test_y_actual_cat2)
report = classification_report(
    test_y_actual_cat2_encoded,
    test_y_pred_cat2_encoded,
    target_names=le_cat2.classes_
)
print(report)

# 전체 정확도
accuracy_cat2 = (test_y_pred_cat2 == test_y_actual_cat2).mean()
print(f"\n🎯 Category2 전체 정확도: {accuracy_cat2:.4f} ({accuracy_cat2*100:.2f}%)")
print(f"평가 데이터: {len(test_y_actual_cat2)}개 모두 평가")

# 7. Category2 예측 샘플 출력
print(f"\n🔍 Category2 예측 샘플 (처음 10개):")
print("-" * 100)

for i in range(min(10, len(test_texts))):
    text = test_texts[i][:50] + "..." if len(test_texts[i]) > 50 else test_texts[i]
    actual_cat1 = test_y_actual_cat1[i]
    pred_cat1 = test_y_pred[i]
    actual_cat2 = test_y_actual_cat2[i]
    pred_cat2 = test_y_pred_cat2[i]
    status_cat2 = "✅" if actual_cat2 == pred_cat2 else "❌"
    
    print(f"{i+1:2d}. {status_cat2} Cat1: {actual_cat1} → {pred_cat1} | Cat2: {actual_cat2:<12} → {pred_cat2:<12}")
    print(f"    텍스트: {text}")
    print()

# 8. Category2 클래스별 성능 요약
print(f"\n📈 Category2 클래스별 성능 요약:")
print("-" * 60)

class_stats_cat2 = defaultdict(lambda: {'total': 0, 'correct': 0})

for actual, pred in zip(test_y_actual_cat2, test_y_pred_cat2):
    class_stats_cat2[actual]['total'] += 1
    if actual == pred:
        class_stats_cat2[actual]['correct'] += 1

print(f"{'클래스':<15} {'전체':<8} {'정답':<8} {'정확도':<10}")
print("-" * 50)

for class_name, stats in sorted(class_stats_cat2.items()):
    if stats['total'] > 0:
        class_accuracy = stats['correct'] / stats['total']
        print(f"{class_name:<15} {stats['total']:<8} {stats['correct']:<8} {class_accuracy:.4f}")

# 9. Category1 vs Category2 성능 비교
print(f"\n📊 최종 성능 비교:")
print("=" * 60)
print(f"Category1 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Category2 정확도: {accuracy_cat2:.4f} ({accuracy_cat2*100:.2f}%)")

if accuracy > accuracy_cat2:
    print("✅ Category1 분류가 더 정확합니다.")
else:
    print("✅ Category2 분류가 더 정확합니다.")


🎯 Category2 모델의 test_data 성능 평가
Category2 학습용 변수들을 재정의합니다...
📊 Category2 데이터 확인:
  - 전체 Category2 데이터: 3359개
  - 고유 Category2 클래스: 64개
  - Category2 클래스 목록: ['갈등', '감동', '걱정', '경멸', '고마움', '고통', '공감', '공포', '궁금함', '귀중함', '그리움', '기대감', '난처함', '날카로움', '냉담', '너그러움', '놀람', '다정함', '답답함', '동정(슬픔)', '두근거림', '만족감', '매력적', '무기력', '미안함', '반가움', '반감', '발열', '부끄러움', '불만', '불신감', '불쾌', '불편함', '비위상함', '사나움', '수치심', '시기심', '신뢰감', '신명남', '실망', '싫증', '심심함', '아쉬움', '아픔', '안정감', '억울함', '외로움', '외면', '욕심', '원망', '위축감', '자랑스러움', '자신감', '절망', '죄책감', '즐거움', '초조함', '치사함', '타오름', '통쾌함', '편안함', '허망', '호감', '후회']
✅ Category2에서 중립이 성공적으로 제거되었습니다.
  - 인코딩된 Category2 클래스: 64개
X shape: (3359, 1024)
y shape: (3359,)
y_cat1_onehot shape: (3359, 10)
X_combined shape: (3359, 1034)
y_cat2 shape: (3359,)
y_cat2_encoded shape: (3359,)

📝 Category2 예측을 위한 데이터 준비...
테스트 데이터:
- Category1 실제값: 664개
- Category2 실제값: 664개
- 테스트 Category2 클래스: 64개
- 테스트 데이터 중립: Category1=5개, Category2=0개

🔧 Category1 예측값으로 Category2 예측용 특성 생성...


c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [30]:
for context in test_data[test_data['category2']=='불만']['context']:
  print(context)

얘들아 딸기축제인가 딸기장례식인가는 뭐 가지마라
어우 별로를 넘어서 아깝다 그냥
핸드폰 바꿨는데.. 어찌나 전뇌이식이 잘되는지 이전 폰에서 듣던 음악 멈춰둔 부분까지 살려놔서 새로 산 기분이 전혀 안남..
친구가 만들어줬어 어이없어서 받자마자 오열함
아니 맞긴한데
솔직히 주말 껴서 4일…연휴라기엔 너무 눈속임임…사실상 이틀 쉰 거잖아
소인 편의점에서 매일우유 크림빵을 만원 어치 사려고 갔는데 3개를 사기에는 부족한 돈이라 슬펐소이다. 물가가 너무 비싼것 같소.
연휴 이틀이 다 주말에 겹쳐져 있었는데 왜 대체 공휴일은 하루만 주는거야. 부족해, 주말 상관없이 설 연휴 3일 다 완벽하게 보장해 줘.
이렇게 자극적인 드라마 아이들도 다 접할텐데 이젠 수위조절도 안하고 막찍는거같아서 좀 안타까움이 생기네요… ㅠㅠ
이천 햅살 커피프라프치노~! 왠지 고소한 커피맛일꺼란 기대를 엄청안고 주문했는데.. 자그마치 6300원ㅜㅜ 비싼 금액인데 만족스럽진 못했다눈ㅜㅜ


In [11]:
# 10. 통합 예측 파이프라인 구현

class EmotionClassificationPipeline:
    """
    텍스트 입력 → Category1 예측 → Category2 예측 → 종합 평가 파이프라인
    """
    
    def __init__(self, embeddings_model, cat1_model, cat2_model, 
                 cat1_encoder, cat2_encoder, cat1_onehot_encoder):
        self.embeddings_model = embeddings_model
        self.cat1_model = cat1_model
        self.cat2_model = cat2_model
        self.cat1_encoder = cat1_encoder
        self.cat2_encoder = cat2_encoder
        self.cat1_onehot_encoder = cat1_onehot_encoder
        
    def predict_single(self, text):
        """
        단일 텍스트에 대해 카테고리1과 카테고리2를 예측
        
        Args:
            text (str): 예측할 텍스트
        
        Returns:
            dict: 예측 결과 딕셔너리
        """
        # 1. 텍스트 임베딩
        text_vector = self.embeddings_model.encode(text).reshape(1, -1)
        
        # 2. Category1 예측
        cat1_pred_encoded = self.cat1_model.predict(text_vector)[0]
        cat1_pred = self.cat1_encoder.inverse_transform([cat1_pred_encoded])[0]
        cat1_prob = self.cat1_model.predict_proba(text_vector)[0].max()
        
        # 3. Category1 예측값을 사용하여 Category2 예측용 특성 생성
        cat1_onehot = self.cat1_onehot_encoder.transform([[cat1_pred]])
        combined_features = np.hstack([text_vector, cat1_onehot])
        
        # 4. Category2 예측
        cat2_pred_encoded = self.cat2_model.predict(combined_features)[0]
        cat2_pred = self.cat2_encoder.inverse_transform([cat2_pred_encoded])[0]
        cat2_prob = self.cat2_model.predict_proba(combined_features)[0].max()
        
        return {
            'text': text,
            'category1_predicted': cat1_pred,
            'category1_confidence': cat1_prob,
            'category2_predicted': cat2_pred,
            'category2_confidence': cat2_prob
        }
    
    def predict_batch(self, texts):
        """
        여러 텍스트에 대해 배치 예측
        
        Args:
            texts (list): 예측할 텍스트 리스트
        
        Returns:
            list: 예측 결과 리스트
        """
        results = []
        for text in texts:
            result = self.predict_single(text)
            results.append(result)
        return results
    
    def evaluate_with_ground_truth(self, texts, true_cat1, true_cat2):
        """
        실제 정답과 비교하여 성능 평가
        두 카테고리가 모두 맞은 경우만 정답으로 처리
        
        Args:
            texts (list): 예측할 텍스트 리스트
            true_cat1 (list): 실제 category1 라벨
            true_cat2 (list): 실제 category2 라벨
        
        Returns:
            dict: 평가 결과
        """
        predictions = self.predict_batch(texts)
        
        total_count = len(texts)
        cat1_correct = 0
        cat2_correct = 0
        both_correct = 0
        
        detailed_results = []
        
        for i, (pred, actual_cat1, actual_cat2) in enumerate(zip(predictions, true_cat1, true_cat2)):
            cat1_match = pred['category1_predicted'] == actual_cat1
            cat2_match = pred['category2_predicted'] == actual_cat2
            both_match = cat1_match and cat2_match
            
            if cat1_match:
                cat1_correct += 1
            if cat2_match:
                cat2_correct += 1
            if both_match:
                both_correct += 1
            
            detailed_results.append({
                'index': i,
                'text': pred['text'],
                'actual_cat1': actual_cat1,
                'predicted_cat1': pred['category1_predicted'],
                'cat1_match': cat1_match,
                'cat1_confidence': pred['category1_confidence'],
                'actual_cat2': actual_cat2,
                'predicted_cat2': pred['category2_predicted'],
                'cat2_match': cat2_match,
                'cat2_confidence': pred['category2_confidence'],
                'both_correct': both_match
            })
        
        return {
            'total_samples': total_count,
            'category1_accuracy': cat1_correct / total_count,
            'category2_accuracy': cat2_correct / total_count,
            'both_correct_accuracy': both_correct / total_count,  # 핵심 지표
            'category1_correct_count': cat1_correct,
            'category2_correct_count': cat2_correct,
            'both_correct_count': both_correct,
            'detailed_results': detailed_results
        }

# 변수 초기화 확인 및 파이프라인 객체 생성
print("🚀 감정 분류 파이프라인 초기화...")

# 필요한 변수들이 정의되었는지 확인
required_vars = ['final_cat1_model', 'final_cat2_model', 'le', 'le_cat2', 'cat1_encoder']
missing_vars = [var for var in required_vars if var not in locals()]

if missing_vars:
    print(f"⚠️ 다음 변수들이 정의되지 않았습니다: {missing_vars}")
    print("모델을 먼저 학습시켜주세요.")
else:
    pipeline = EmotionClassificationPipeline(
        embeddings_model=embeddings_model,
        cat1_model=final_cat1_model,
        cat2_model=final_cat2_model,
        cat1_encoder=le,
        cat2_encoder=le_cat2,
        cat1_onehot_encoder=cat1_encoder
    )
    print("✅ 파이프라인 초기화 완료!")

🚀 감정 분류 파이프라인 초기화...
✅ 파이프라인 초기화 완료!


In [12]:
# 11. 파이프라인으로 test_data 평가 (두 카테고리 모두 맞은 경우만 정답 처리)

print("🎯 파이프라인으로 test_data 종합 평가")
print("="*80)

# test_data 준비
test_texts = test_data['context'].fillna('').astype(str).tolist()
test_true_cat1 = test_data['category1'].values
test_true_cat2 = test_data['category2'].values

print(f"평가 데이터: {len(test_texts)}개")
print(f"Category1 클래스 수: {len(np.unique(test_true_cat1))}")
print(f"Category2 클래스 수: {len(np.unique(test_true_cat2))}")

# 중립 데이터 확인
neutral_cat1_count = (test_true_cat1 == '중립').sum()
neutral_cat2_count = (test_true_cat2 == '중립').sum()
print(f"테스트 데이터 중립: Category1={neutral_cat1_count}개, Category2={neutral_cat2_count}개")

# 파이프라인으로 종합 평가 실행 (모든 데이터 사용 - Category1에 중립 포함)
print(f"\n🔄 파이프라인 평가 실행 중...")
print(f"Category1: 중립 포함하여 평가")
print(f"Category2: 중립 없음 (테스트 데이터에도 없음)")

evaluation_results = pipeline.evaluate_with_ground_truth(
    test_texts, test_true_cat1, test_true_cat2
)

# 결과 출력
print(f"\n📊 파이프라인 종합 평가 결과:")
print("="*60)
print(f"전체 테스트 샘플 수: {evaluation_results['total_samples']}")
print(f"")
print(f"📈 개별 정확도:")
print(f"  Category1 정확도: {evaluation_results['category1_accuracy']:.4f} ({evaluation_results['category1_correct_count']}/{evaluation_results['total_samples']})")
print(f"  Category2 정확도: {evaluation_results['category2_accuracy']:.4f} ({evaluation_results['category2_correct_count']}/{evaluation_results['total_samples']})")
print(f"")
print(f"🎯 핵심 지표 - 두 카테고리 모두 정답:")
print(f"  종합 정확도: {evaluation_results['both_correct_accuracy']:.4f} ({evaluation_results['both_correct_count']}/{evaluation_results['total_samples']})")
print(f"  종합 정확도: {evaluation_results['both_correct_accuracy']*100:.2f}%")

# 상세 분석
print(f"\n🔍 상세 분석:")
print("-"*60)

# 카테고리별 매치 패턴 분석
both_correct = sum(1 for r in evaluation_results['detailed_results'] if r['both_correct'])
only_cat1_correct = sum(1 for r in evaluation_results['detailed_results'] if r['cat1_match'] and not r['cat2_match'])
only_cat2_correct = sum(1 for r in evaluation_results['detailed_results'] if not r['cat1_match'] and r['cat2_match'])
both_wrong = sum(1 for r in evaluation_results['detailed_results'] if not r['cat1_match'] and not r['cat2_match'])

print(f"두 카테고리 모두 정답: {both_correct}개 ({both_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"Category1만 정답: {only_cat1_correct}개 ({only_cat1_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"Category2만 정답: {only_cat2_correct}개 ({only_cat2_correct/evaluation_results['total_samples']*100:.2f}%)")
print(f"둘 다 틀림: {both_wrong}개 ({both_wrong/evaluation_results['total_samples']*100:.2f}%)")

# 샘플 출력 - 두 카테고리 모두 맞은 케이스
print(f"\n✅ 두 카테고리 모두 정답인 샘플들 (처음 10개):")
print("="*100)

correct_samples = [r for r in evaluation_results['detailed_results'] if r['both_correct']]
for i, result in enumerate(correct_samples[:10]):
    text = result['text'][:60] + "..." if len(result['text']) > 60 else result['text']
    print(f"{i+1:2d}. Cat1: {result['actual_cat1']} ✓ | Cat2: {result['actual_cat2']} ✓")
    print(f"    신뢰도: Cat1={result['cat1_confidence']:.3f}, Cat2={result['cat2_confidence']:.3f}")
    print(f"    텍스트: {text}")
    print()

# 샘플 출력 - 둘 다 틀린 케이스  
print(f"\n❌ 두 카테고리 모두 틀린 샘플들 (처음 10개):")
print("="*100)

wrong_samples = [r for r in evaluation_results['detailed_results'] if not r['both_correct'] and not r['cat1_match'] and not r['cat2_match']]
for i, result in enumerate(wrong_samples[:10]):
    text = result['text'][:60] + "..." if len(result['text']) > 60 else result['text']
    print(f"{i+1:2d}. Cat1: {result['actual_cat1']} → {result['predicted_cat1']} | Cat2: {result['actual_cat2']} → {result['predicted_cat2']}")
    print(f"    신뢰도: Cat1={result['cat1_confidence']:.3f}, Cat2={result['cat2_confidence']:.3f}")
    print(f"    텍스트: {text}")
    print()

# 중립 데이터 특별 분석
print(f"\n⚖️ 중립 데이터 분석 (Category1):")
print("-"*60)

neutral_results = [r for r in evaluation_results['detailed_results'] if r['actual_cat1'] == '중립']
if len(neutral_results) > 0:
    neutral_correct = sum(1 for r in neutral_results if r['cat1_match'])
    print(f"중립 데이터 총 {len(neutral_results)}개 중 {neutral_correct}개 정답 ({neutral_correct/len(neutral_results)*100:.1f}%)")
    
    print(f"\n중립 샘플 예측 결과 (처음 5개):")
    for i, result in enumerate(neutral_results[:5]):
        text = result['text'][:50] + "..." if len(result['text']) > 50 else result['text']
        status = "✅" if result['cat1_match'] else "❌"
        print(f"{i+1}. {status} 예측: {result['predicted_cat1']} (신뢰도: {result['cat1_confidence']:.3f})")
        print(f"   텍스트: {text}")
else:
    print("테스트 데이터에 중립이 없습니다.")

print(f"\n🔥 결론:")
print(f"이 파이프라인에서 두 카테고리가 모두 정확하게 예측된 경우는 전체의 {evaluation_results['both_correct_accuracy']*100:.2f}%입니다.")
print(f"※ Category1은 중립을 포함하여 평가, Category2는 중립이 없어 일치합니다.")

🎯 파이프라인으로 test_data 종합 평가
평가 데이터: 664개
Category1 클래스 수: 10
Category2 클래스 수: 64
테스트 데이터 중립: Category1=5개, Category2=0개

🔄 파이프라인 평가 실행 중...
Category1: 중립 포함하여 평가
Category2: 중립 없음 (테스트 데이터에도 없음)

📊 파이프라인 종합 평가 결과:
전체 테스트 샘플 수: 664

📈 개별 정확도:
  Category1 정확도: 0.4880 (324/664)
  Category2 정확도: 0.2425 (161/664)

🎯 핵심 지표 - 두 카테고리 모두 정답:
  종합 정확도: 0.2244 (149/664)
  종합 정확도: 22.44%

🔍 상세 분석:
------------------------------------------------------------
두 카테고리 모두 정답: 149개 (22.44%)
Category1만 정답: 175개 (26.36%)
Category2만 정답: 12개 (1.81%)
둘 다 틀림: 328개 (49.40%)

✅ 두 카테고리 모두 정답인 샘플들 (처음 10개):
 1. Cat1: 기쁨 ✓ | Cat2: 만족감 ✓
    신뢰도: Cat1=0.467, Cat2=0.208
    텍스트: 미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드인데, 선택할 수 있는 디...

 2. Cat1: 기쁨 ✓ | Cat2: 만족감 ✓
    신뢰도: Cat1=0.943, Cat2=0.283
    텍스트: 우연히 보게 된 영상인데, 노래가 너무 좋아서 플리에도 추가하고, 카카오톡 프뮤로도 해놨음. 음원도 좋긴 한...

 3. Cat1: 욕망 ✓ | Cat2: 궁금함 ✓
    신뢰도: Cat1=0.653, Cat2=0.422
    텍스트: 일본은 근무시간에 개인메세지 안 한다고??? 신기
애초에 개인 메세지 도구인 카카오톡으로 업무를 하는데 친구...

 4. Cat1: 기

In [13]:
# 12. 새로운 텍스트 예측 데모 함수

def demo_pipeline_prediction():
    """
    새로운 텍스트들에 대해 파이프라인 예측 시연
    """
    print("🚀 파이프라인 예측 시연")
    print("="*80)
    
    # 예제 텍스트들
    demo_texts = [
        "친구가 생일 파티를 준비해줘서 너무 감동받았어",
        "시험 결과가 나쁘게 나와서 정말 실망스럽다", 
        "새로운 직장이 확정되어서 설레고 기대된다",
        "누군가 내 뒷담화를 하는 걸 들어서 화가 난다"
    ]
    
    for i, text in enumerate(demo_texts, 1):
        print(f"\n📝 예제 {i}: {text}")
        print("-"*60)
        
        # 파이프라인으로 예측
        result = pipeline.predict_single(text)
        
        print(f"🎯 예측 결과:")
        print(f"  Category1: {result['category1_predicted']} (신뢰도: {result['category1_confidence']:.3f})")
        print(f"  Category2: {result['category2_predicted']} (신뢰도: {result['category2_confidence']:.3f})")

# 시연 실행
demo_pipeline_prediction()

🚀 파이프라인 예측 시연

📝 예제 1: 친구가 생일 파티를 준비해줘서 너무 감동받았어
------------------------------------------------------------
🎯 예측 결과:
  Category1: 기쁨 (신뢰도: 0.994)
  Category2: 고마움 (신뢰도: 0.495)

📝 예제 2: 시험 결과가 나쁘게 나와서 정말 실망스럽다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 슬픔 (신뢰도: 0.584)
  Category2: 실망 (신뢰도: 0.804)

📝 예제 3: 새로운 직장이 확정되어서 설레고 기대된다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 기쁨 (신뢰도: 0.983)
  Category2: 기대감 (신뢰도: 0.978)

📝 예제 4: 누군가 내 뒷담화를 하는 걸 들어서 화가 난다
------------------------------------------------------------
🎯 예측 결과:
  Category1: 분노 (신뢰도: 0.578)
  Category2: 불쾌 (신뢰도: 0.826)


In [14]:
# 13. PCA 차원 축소 파이프라인 - 두 카테고리 모두 정답 정확도 비교

print("🔍 PCA 차원 축소 파이프라인 평가")
print("=" * 80)

# PCA 차원 리스트 정의
pca_dimensions = [128, 256, 512, 768]

# 결과 저장용 딕셔너리
pca_results = {}

# 원본 벡터 크기 확인
original_dim = X.shape[1]
print(f"원본 벡터 차원: {original_dim}")
print(f"평가할 PCA 차원: {pca_dimensions}")
print(f"훈련 데이터: {X.shape[0]}개, 테스트 데이터: {len(test_texts)}개")
print()

# 각 PCA 차원에 대해 평가
for n_components in pca_dimensions:
    print(f"\n{'='*20} PCA {n_components}차원 평가 {'='*20}")
    
    # 1. PCA 적용
    print(f"📐 PCA {n_components}차원으로 축소 중...")
    pca = PCA(n_components=n_components, random_state=42)
    
    # 훈련 데이터 PCA 변환
    X_pca = pca.fit_transform(X)
    print(f"  훈련 데이터: {X.shape} → {X_pca.shape}")
    
    # 테스트 데이터 PCA 변환
    test_X_pca = pca.transform(test_X)
    print(f"  테스트 데이터: {test_X.shape} → {test_X_pca.shape}")
    
    # 설명 가능한 분산 비율
    explained_variance = pca.explained_variance_ratio_.sum()
    print(f"  설명 가능한 분산 비율: {explained_variance:.4f} ({explained_variance*100:.2f}%)")
    
    # 2. Category1 모델 훈련 (PCA 적용)
    print(f"🤖 Category1 모델 훈련 중...")
    cat1_model_pca = xgb.XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    
    cat1_model_pca.fit(X_pca, y_encoded)
    
    # Category1 예측
    test_y_pred_cat1_pca_encoded = cat1_model_pca.predict(test_X_pca)
    test_y_pred_cat1_pca = le.inverse_transform(test_y_pred_cat1_pca_encoded)
    
    # Category1 정확도
    cat1_accuracy_pca = (test_y_pred_cat1_pca == test_y_actual).mean()
    print(f"  Category1 정확도: {cat1_accuracy_pca:.4f} ({cat1_accuracy_pca*100:.2f}%)")
    
    # 3. Category2 모델 훈련 (PCA 적용)
    print(f"🤖 Category2 모델 훈련 중...")
    
    # Category1 원핫 인코딩 (PCA 적용된 예측값 사용)
    cat1_onehot_pca = cat1_encoder.transform(test_y_pred_cat1_pca.reshape(-1, 1))
    
    # PCA 적용된 벡터와 Category1 원핫 결합 (훈련용)
    y_cat1_onehot_pca = cat1_encoder.transform(y.reshape(-1, 1))
    X_combined_pca = np.hstack([X_pca, y_cat1_onehot_pca])
    
    # PCA 적용된 벡터와 Category1 예측 원핫 결합 (테스트용)
    test_X_combined_pca = np.hstack([test_X_pca, cat1_onehot_pca])
    
    cat2_model_pca = xgb.XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    
    cat2_model_pca.fit(X_combined_pca, y_cat2_encoded)
    
    # Category2 예측
    test_y_pred_cat2_pca_encoded = cat2_model_pca.predict(test_X_combined_pca)
    test_y_pred_cat2_pca = le_cat2.inverse_transform(test_y_pred_cat2_pca_encoded)
    
    # Category2 정확도
    cat2_accuracy_pca = (test_y_pred_cat2_pca == test_y_actual_cat2).mean()
    print(f"  Category2 정확도: {cat2_accuracy_pca:.4f} ({cat2_accuracy_pca*100:.2f}%)")
    
    # 4. 두 카테고리 모두 정답인 경우 계산
    both_correct_pca = (test_y_pred_cat1_pca == test_y_actual) & (test_y_pred_cat2_pca == test_y_actual_cat2)
    both_accuracy_pca = both_correct_pca.mean()
    both_count_pca = both_correct_pca.sum()
    
    print(f"  🎯 두 카테고리 모두 정답: {both_count_pca}/{len(test_y_actual)} ({both_accuracy_pca:.4f}, {both_accuracy_pca*100:.2f}%)")
    
    # 결과 저장
    pca_results[n_components] = {
        'explained_variance': explained_variance,
        'cat1_accuracy': cat1_accuracy_pca,
        'cat2_accuracy': cat2_accuracy_pca,
        'both_accuracy': both_accuracy_pca,
        'both_count': both_count_pca,
        'cat1_predictions': test_y_pred_cat1_pca,
        'cat2_predictions': test_y_pred_cat2_pca
    }

# 5. 전체 결과 비교 분석
print(f"\n{'='*25} 결과 비교 분석 {'='*25}")
print(f"{'PCA 차원':<10} {'분산비율':<12} {'Cat1 정확도':<12} {'Cat2 정확도':<12} {'종합 정확도':<12}")
print("-" * 70)

# 원본 결과 (PCA 없음) 계산
original_both_correct = (test_y_pred == test_y_actual) & (test_y_pred_cat2 == test_y_actual_cat2)
original_both_accuracy = original_both_correct.mean()

print(f"{'원본':<10} {'100.00%':<12} {accuracy:<12.4f} {accuracy_cat2:<12.4f} {original_both_accuracy:<12.4f}")

# PCA 결과들
for dim in pca_dimensions:
    result = pca_results[dim]
    print(f"{dim:<10} {result['explained_variance']*100:<11.2f}% {result['cat1_accuracy']:<12.4f} {result['cat2_accuracy']:<12.4f} {result['both_accuracy']:<12.4f}")

# 6. 최고 성능 차원 찾기
print(f"\n🏆 성능 분석:")
best_both_accuracy = max(result['both_accuracy'] for result in pca_results.values())
best_pca_dim = max(pca_results.keys(), key=lambda k: pca_results[k]['both_accuracy'])

print(f"  원본 (1024차원) 종합 정확도: {original_both_accuracy:.4f} ({original_both_accuracy*100:.2f}%)")
print(f"  최고 PCA ({best_pca_dim}차원) 종합 정확도: {best_both_accuracy:.4f} ({best_both_accuracy*100:.2f}%)")

if best_both_accuracy > original_both_accuracy:
    improvement = (best_both_accuracy - original_both_accuracy) * 100
    print(f"  ✅ PCA {best_pca_dim}차원이 원본보다 {improvement:.2f}%p 더 좋습니다!")
else:
    decline = (original_both_accuracy - best_both_accuracy) * 100
    print(f"  ⚠️ 원본이 최고 PCA보다 {decline:.2f}%p 더 좋습니다.")

# 7. 차원별 성능 변화 시각화 (텍스트)
print(f"\n📊 차원별 성능 변화:")
print(f"PCA 차원  → 종합 정확도")
print("-" * 25)
for dim in sorted(pca_dimensions):
    result = pca_results[dim]
    bar_length = int(result['both_accuracy'] * 50)  # 50칸 기준
    bar = "█" * bar_length + "░" * (50 - bar_length)
    print(f"{dim:>3}차원   → {result['both_accuracy']:.3f} |{bar}|")

# 8. 상세 샘플 분석 (최고 성능 PCA 차원)
print(f"\n🔍 최고 성능 PCA {best_pca_dim}차원 상세 분석:")
print("-" * 80)

best_cat1_preds = pca_results[best_pca_dim]['cat1_predictions']
best_cat2_preds = pca_results[best_pca_dim]['cat2_predictions']

# 원본 vs PCA 예측 비교
agreement_count = 0
disagreement_examples = []

for i in range(min(len(test_y_actual), 20)):  # 처음 20개 샘플만
    original_both = (test_y_pred[i] == test_y_actual[i]) and (test_y_pred_cat2[i] == test_y_actual_cat2[i])
    pca_both = (best_cat1_preds[i] == test_y_actual[i]) and (best_cat2_preds[i] == test_y_actual_cat2[i])
    
    if original_both == pca_both:
        agreement_count += 1
    else:
        disagreement_examples.append({
            'index': i,
            'text': test_texts[i][:60] + "..." if len(test_texts[i]) > 60 else test_texts[i],
            'actual_cat1': test_y_actual[i],
            'actual_cat2': test_y_actual_cat2[i],
            'original_both': original_both,
            'pca_both': pca_both
        })

print(f"처음 20개 샘플에서 원본과 PCA 예측 일치율: {agreement_count}/20 ({agreement_count/20*100:.1f}%)")

if disagreement_examples:
    print(f"\n원본 vs PCA 예측이 다른 샘플들 (처음 5개):")
    for i, example in enumerate(disagreement_examples[:5]):
        status_original = "✅" if example['original_both'] else "❌"
        status_pca = "✅" if example['pca_both'] else "❌"
        print(f"{i+1}. 원본: {status_original} | PCA: {status_pca}")
        print(f"   실제: Cat1={example['actual_cat1']}, Cat2={example['actual_cat2']}")
        print(f"   텍스트: {example['text']}")

print(f"\n💡 결론: PCA를 통한 차원 축소는 ")
if best_both_accuracy > original_both_accuracy:
    print("성능 향상에 도움이 됩니다. 계산 효율성과 성능을 모두 개선할 수 있습니다.")
else:
    print("성능을 약간 저하시키지만, 계산 효율성은 크게 개선됩니다.")

🔍 PCA 차원 축소 파이프라인 평가
원본 벡터 차원: 1024
평가할 PCA 차원: [128, 256, 512, 768]
훈련 데이터: 3359개, 테스트 데이터: 664개


==================== PCA 128차원 평가 ====================
📐 PCA 128차원으로 축소 중...
  훈련 데이터: (3359, 1024) → (3359, 128)
  테스트 데이터: (664, 1024) → (664, 128)
  설명 가능한 분산 비율: 0.8185 (81.85%)
🤖 Category1 모델 훈련 중...
  Category1 정확도: 0.5015 (50.15%)
🤖 Category2 모델 훈련 중...
  Category2 정확도: 0.2485 (24.85%)
  🎯 두 카테고리 모두 정답: 152/664 (0.2289, 22.89%)

==================== PCA 256차원 평가 ====================
📐 PCA 256차원으로 축소 중...
  훈련 데이터: (3359, 1024) → (3359, 256)
  테스트 데이터: (664, 1024) → (664, 256)
  설명 가능한 분산 비율: 0.9409 (94.09%)
🤖 Category1 모델 훈련 중...
  Category1 정확도: 0.4880 (48.80%)
🤖 Category2 모델 훈련 중...
  Category2 정확도: 0.2199 (21.99%)
  🎯 두 카테고리 모두 정답: 133/664 (0.2003, 20.03%)

==================== PCA 512차원 평가 ====================
📐 PCA 512차원으로 축소 중...
  훈련 데이터: (3359, 1024) → (3359, 512)
  테스트 데이터: (664, 1024) → (664, 512)
  설명 가능한 분산 비율: 0.9961 (99.61%)
🤖 Category1 모델 훈련 중...
  Category1 정확도: 0.

In [16]:
# 14. AutoGluon 개별 모델 - Category1 & Category2 각각 최적 모델 훈련 및 평가

print("🚀 AutoGluon으로 Category1 & Category2 개별 최적 모델 탐색")
print("="*80)

try:
    from autogluon.tabular import TabularPredictor
    print("✅ AutoGluon 라이브러리 로드 성공")
    autogluon_available = True
except ImportError:
    print("❌ AutoGluon이 설치되지 않았습니다. 다음 명령으로 설치하세요:")
    print("pip install autogluon")
    autogluon_available = False
    
import os
import warnings
warnings.filterwarnings('ignore')

# 1. AutoGluon용 데이터 준비
print("\n📊 AutoGluon용 데이터 준비...")

# 훈련 데이터: 벡터만 사용 (라벨 제외)
train_features_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
train_features_df['category1'] = y  # re_category1 사용
train_features_df['category2'] = data['re_category2'].values

# 테스트 데이터: 벡터만
test_features_df = pd.DataFrame(test_X, columns=[f'feature_{i}' for i in range(test_X.shape[1])])

print(f"훈련 데이터 크기: {train_features_df.shape}")
print(f"테스트 데이터 크기: {test_features_df.shape}")
print(f"Category1 클래스 수: {len(train_features_df['category1'].unique())}")
print(f"Category2 클래스 수: {len(train_features_df['category2'].unique())}")

# 모델 저장 경로
cat1_save_path = "autogluon_category1_model"
cat2_save_path = "autogluon_category2_model"

# 기존 모델 폴더 삭제
for path in [cat1_save_path, cat2_save_path]:
    if os.path.exists(path):
        import shutil
        shutil.rmtree(path)
        print(f"기존 모델 폴더 {path} 삭제됨")

# 2. Category1 모델 훈련
print(f"\n🤖 Category1 AutoGluon 모델 훈련 시작...")
print("="*60)

cat1_predictor = None
if autogluon_available:
    try:
        # Category1 예측기 생성
        cat1_predictor = TabularPredictor(
            label='category1',
            path=cat1_save_path,
            eval_metric='f1_macro',
            problem_type='multiclass'
        )
        
        # 모델 훈련
        print("Category1 모델들 훈련 중... (예상 소요시간: 3-5분)")
        cat1_predictor = cat1_predictor.fit(
            train_data=train_features_df.drop(['category2'], axis=1),
            time_limit=180,  # 3분
            presets='medium_quality_faster_train',
            num_bag_folds=3,
            num_bag_sets=1,
            num_stack_levels=1
        )
        
        print("✅ Category1 AutoGluon 모델 훈련 완료!")
        
        # Category1 리더보드
        print(f"\n📋 Category1 모델 리더보드:")
        cat1_leaderboard = cat1_predictor.leaderboard()
        print(cat1_leaderboard.head())
        
        cat1_best_model = cat1_leaderboard.iloc[0]['model']
        cat1_best_score = cat1_leaderboard.iloc[0]['score_val']
        print(f"🏆 Category1 최고 모델: {cat1_best_model} (F1: {cat1_best_score:.4f})")
        
    except Exception as e:
        print(f"❌ Category1 AutoGluon 훈련 실패: {str(e)}")
        cat1_predictor = None

# 3. Category2 모델 훈련
print(f"\n🤖 Category2 AutoGluon 모델 훈련 시작...")
print("="*60)

cat2_predictor = None
if autogluon_available:
    try:
        # Category2 예측기 생성
        cat2_predictor = TabularPredictor(
            label='category2',
            path=cat2_save_path,
            eval_metric='f1_macro',
            problem_type='multiclass'
        )
        
        # Category1 예측값을 특성으로 추가 (기존 파이프라인과 동일한 방식)
        if cat1_predictor is not None:
            # 훈련 데이터에 Category1 예측값 추가
            train_cat1_pred = cat1_predictor.predict(train_features_df.drop(['category1', 'category2'], axis=1))
            train_enhanced_df = train_features_df.drop(['category1'], axis=1).copy()
            train_enhanced_df['predicted_category1'] = train_cat1_pred
        else:
            # Category1 예측기가 없으면 실제 Category1 값 사용
            train_enhanced_df = train_features_df.drop(['category1'], axis=1).copy()
            train_enhanced_df['predicted_category1'] = train_features_df['category1']
        
        # 모델 훈련
        print("Category2 모델들 훈련 중... (예상 소요시간: 3-5분)")
        cat2_predictor = cat2_predictor.fit(
            train_data=train_enhanced_df,
            time_limit=180,  # 3분
            presets='medium_quality_faster_train',
            num_bag_folds=3,
            num_bag_sets=1,
            num_stack_levels=1
        )
        
        print("✅ Category2 AutoGluon 모델 훈련 완료!")
        
        # Category2 리더보드
        print(f"\n📋 Category2 모델 리더보드:")
        cat2_leaderboard = cat2_predictor.leaderboard()
        print(cat2_leaderboard.head())
        
        cat2_best_model = cat2_leaderboard.iloc[0]['model']
        cat2_best_score = cat2_leaderboard.iloc[0]['score_val']
        print(f"🏆 Category2 최고 모델: {cat2_best_model} (F1: {cat2_best_score:.4f})")
        
    except Exception as e:
        print(f"❌ Category2 AutoGluon 훈련 실패: {str(e)}")
        cat2_predictor = None

# 4. XGBoost 대안 모델 (AutoGluon 실패시)
if not autogluon_available or cat1_predictor is None or cat2_predictor is None:
    print(f"\n🔄 XGBoost 대안 모델로 진행...")
    
    # Category1 XGBoost 모델
    print("Category1 XGBoost 모델 훈련...")
    cat1_xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    cat1_xgb_model.fit(X, y_encoded)
    
    # Category2 XGBoost 모델 (Category1 포함)
    print("Category2 XGBoost 모델 훈련...")
    y_cat1_onehot_xgb = cat1_encoder.transform(y.reshape(-1, 1))
    X_combined_xgb = np.hstack([X, y_cat1_onehot_xgb])
    
    cat2_xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    cat2_xgb_model.fit(X_combined_xgb, y_cat2_encoded)
    
    print("✅ XGBoost 대안 모델 훈련 완료!")

# 5. 모델 로드 및 테스트 예측 시뮬레이션
print(f"\n🔄 저장된 모델 로드 및 테스트 예측")
print("="*60)

# 실제로는 다음과 같이 로드할 수 있습니다:
# cat1_loaded_predictor = TabularPredictor.load(cat1_save_path)
# cat2_loaded_predictor = TabularPredictor.load(cat2_save_path)

if autogluon_available and cat1_predictor is not None and cat2_predictor is not None:
    print("AutoGluon 모델로 예측 수행...")
    
    # Category1 예측
    print("📈 Category1 예측 중...")
    test_pred_cat1_ag = cat1_predictor.predict(test_features_df)
    test_pred_cat1_proba = cat1_predictor.predict_proba(test_features_df)
    
    # Category2 예측 (Category1 예측값 포함)
    print("📈 Category2 예측 중...")
    test_enhanced_df = test_features_df.copy()
    test_enhanced_df['predicted_category1'] = test_pred_cat1_ag
    test_pred_cat2_ag = cat2_predictor.predict(test_enhanced_df)
    test_pred_cat2_proba = cat2_predictor.predict_proba(test_enhanced_df)
    
    model_type = "AutoGluon"
    
else:
    print("XGBoost 모델로 예측 수행...")
    
    # Category1 예측 (XGBoost)
    test_pred_cat1_encoded_ag = cat1_xgb_model.predict(test_X)
    test_pred_cat1_ag = le.inverse_transform(test_pred_cat1_encoded_ag)
    
    # Category2 예측 (XGBoost)
    test_cat1_onehot_ag = cat1_encoder.transform(test_pred_cat1_ag.reshape(-1, 1))
    test_X_combined_ag = np.hstack([test_X, test_cat1_onehot_ag])
    test_pred_cat2_encoded_ag = cat2_xgb_model.predict(test_X_combined_ag)
    test_pred_cat2_ag = le_cat2.inverse_transform(test_pred_cat2_encoded_ag)
    
    model_type = "XGBoost 고성능"

print(f"✅ {model_type} 모델 예측 완료!")

# 6. 성능 평가 - 기존 파이프라인과 동일한 방식
print(f"\n📊 {model_type} 모델 성능 평가")
print("="*60)

# 개별 카테고리 정확도
cat1_accuracy_ag = (test_pred_cat1_ag == test_y_actual).mean()
cat2_accuracy_ag = (test_pred_cat2_ag == test_y_actual_cat2).mean()

print(f"Category1 정확도: {cat1_accuracy_ag:.4f} ({cat1_accuracy_ag*100:.2f}%)")
print(f"Category2 정확도: {cat2_accuracy_ag:.4f} ({cat2_accuracy_ag*100:.2f}%)")

# 🎯 핵심 지표: 두 카테고리 모두 정답
both_correct_ag = (test_pred_cat1_ag == test_y_actual) & (test_pred_cat2_ag == test_y_actual_cat2)
both_accuracy_ag = both_correct_ag.mean()
both_count_ag = both_correct_ag.sum()

print(f"\n🎯 핵심 지표 - 두 카테고리 모두 정답:")
print(f"정확도: {both_accuracy_ag:.4f} ({both_accuracy_ag*100:.2f}%)")
print(f"정답 개수: {both_count_ag}/{len(test_y_actual)}")

# 7. 기존 파이프라인과 성능 비교
print(f"\n📈 모델 성능 비교:")
print("-"*60)
print(f"{'모델':<20} {'Cat1 정확도':<12} {'Cat2 정확도':<12} {'종합 정확도':<12} {'개선':<10}")
print("-"*60)

# 기존 결과 계산
original_both_correct = (test_y_pred == test_y_actual) & (test_y_pred_cat2 == test_y_actual_cat2)
original_both_accuracy = original_both_correct.mean()

print(f"{'기존 XGBoost':<20} {accuracy:<12.4f} {accuracy_cat2:<12.4f} {original_both_accuracy:<12.4f} {'기준':<10}")

# AutoGluon/XGBoost 고성능 결과
improvement = both_accuracy_ag - original_both_accuracy
improvement_text = f"+{improvement:.4f}" if improvement > 0 else f"{improvement:.4f}"

print(f"{model_type:<20} {cat1_accuracy_ag:<12.4f} {cat2_accuracy_ag:<12.4f} {both_accuracy_ag:<12.4f} {improvement_text:<10}")

# 8. 상세 분류 성능 분석
print(f"\n📋 상세 분류 성능 (Classification Report)")
print("="*80)

# Category1 분류 리포트
from sklearn.metrics import classification_report, f1_score
print("🔹 Category1 분류 성능:")
cat1_f1 = f1_score(test_y_actual, test_pred_cat1_ag, average='macro')
print(f"F1-Score (macro): {cat1_f1:.4f}")
cat1_report = classification_report(test_y_actual, test_pred_cat1_ag)
print(cat1_report)

print("\n🔹 Category2 분류 성능:")
cat2_f1 = f1_score(test_y_actual_cat2, test_pred_cat2_ag, average='macro')
print(f"F1-Score (macro): {cat2_f1:.4f}")
cat2_report = classification_report(test_y_actual_cat2, test_pred_cat2_ag)
print(cat2_report)

# 9. 예측 샘플 분석
print(f"\n🔍 예측 샘플 분석 (처음 15개):")
print("="*100)

for i in range(min(15, len(test_y_actual))):
    text = test_texts[i][:50] + "..." if len(test_texts[i]) > 50 else test_texts[i]
    
    actual_cat1 = test_y_actual[i]
    actual_cat2 = test_y_actual_cat2[i]
    pred_cat1 = test_pred_cat1_ag[i]
    pred_cat2 = test_pred_cat2_ag[i]
    
    cat1_match = actual_cat1 == pred_cat1
    cat2_match = actual_cat2 == pred_cat2
    both_match = cat1_match and cat2_match
    
    status = "✅" if both_match else "❌"
    cat1_status = "✓" if cat1_match else "✗"
    cat2_status = "✓" if cat2_match else "✗"
    
    print(f"{i+1:2d}. {status} Cat1: {actual_cat1} → {pred_cat1} {cat1_status} | Cat2: {actual_cat2} → {pred_cat2} {cat2_status}")
    print(f"      텍스트: {text}")
    print()

# 10. 결론 및 모델 저장 정보
print(f"\n💡 결론 및 권장사항:")
print("="*60)

if improvement > 0.01:
    print(f"✅ {model_type} 모델이 기존보다 {improvement*100:.2f}%p 향상되었습니다!")
    print("권장사항: 새로운 모델을 사용하세요.")
elif improvement > -0.01:
    print(f"📊 {model_type} 모델과 기존 모델이 유사합니다 (차이: {abs(improvement)*100:.2f}%p).")
    print("권장사항: 계산 효율성을 고려하여 선택하세요.")
else:
    print(f"⚠️ 기존 모델이 {abs(improvement)*100:.2f}%p 더 좋습니다.")
    print("권장사항: 기존 모델을 유지하거나 하이퍼파라미터를 조정하세요.")

if autogluon_available and cat1_predictor is not None and cat2_predictor is not None:
    print(f"\n💾 AutoGluon 모델 저장 위치:")
    print(f"- Category1: {cat1_save_path}")
    print(f"- Category2: {cat2_save_path}")
    print(f"\n📥 모델 로드 방법:")
    print(f"from autogluon.tabular import TabularPredictor")
    print(f"cat1_predictor = TabularPredictor.load('{cat1_save_path}')")
    print(f"cat2_predictor = TabularPredictor.load('{cat2_save_path}')")
    print(f"\n🔮 새로운 데이터 예측:")
    print(f"cat1_pred = cat1_predictor.predict(new_data)")
    print(f"new_data_with_cat1 = new_data.copy()")
    print(f"new_data_with_cat1['predicted_category1'] = cat1_pred")
    print(f"cat2_pred = cat2_predictor.predict(new_data_with_cat1)")
else:
    print(f"\n⚙️ XGBoost 모델이 메모리에 저장되어 있습니다.")
    print(f"필요시 pickle을 사용하여 저장할 수 있습니다.")

print(f"\n🎯 최종 성과:")
print(f"두 카테고리 동시 정답률: {both_accuracy_ag*100:.2f}% ({both_count_ag}/{len(test_y_actual)}개)")

Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       15.70 GB / 31.91 GB (49.2%)
Disk Space Avail:   89.07 GB / 465.12 GB (19.1%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'


🚀 AutoGluon으로 Category1 & Category2 개별 최적 모델 탐색
✅ AutoGluon 라이브러리 로드 성공

📊 AutoGluon용 데이터 준비...
훈련 데이터 크기: (3359, 1026)
테스트 데이터 크기: (664, 1024)
Category1 클래스 수: 10
Category2 클래스 수: 64

🤖 Category1 AutoGluon 모델 훈련 시작...
Category1 모델들 훈련 중... (예상 소요시간: 3-5분)


Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "c:\Users\user\Desktop\SKN_AFTER_STUDY\autogluon_category1_model"
Train Data Rows:    3359
Train Data Columns: 1024
Label Column:       category1
Problem Type:       multiclass
Preprocessing data ...
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9994045846978268
Train Data Class Count: 9
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    16070.75 MB
	Train Data (Original)  Memory Usage: 26.23 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGen

✅ Category1 AutoGluon 모델 훈련 완료!

📋 Category1 모델 리더보드:
                    model  score_val eval_metric  pred_time_val    fit_time  \
0     WeightedEnsemble_L3   0.845421    f1_macro       0.645323  152.975757   
1       LightGBMXT_BAG_L2   0.835284    f1_macro       0.449279  138.235540   
2     WeightedEnsemble_L2   0.834770    f1_macro       0.367259  107.058965   
3  NeuralNetFastAI_BAG_L2   0.834273    f1_macro       0.558301  121.429098   
4  NeuralNetFastAI_BAG_L1   0.824998    f1_macro       0.156036   11.753919   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.002001           0.207047            3       True   
1                0.085021          31.339612            2       True   
2                0.003001           0.163037            2       True   
3                0.194043          14.533170            2       True   
4                0.156036          11.753919            1       True   

   fit_order  
0          8  
1       

Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       14.82 GB / 31.91 GB (46.4%)
Disk Space Avail:   88.98 GB / 465.12 GB (19.1%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'


Category2 모델들 훈련 중... (예상 소요시간: 3-5분)


Beginning AutoGluon training ... Time limit = 180s
AutoGluon will save models to "c:\Users\user\Desktop\SKN_AFTER_STUDY\autogluon_category2_model"
Train Data Rows:    3359
Train Data Columns: 1025
Label Column:       category2
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 64
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15207.55 MB
	Train Data (Original)  Memory Usage: 26.49 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Gen

✅ Category2 AutoGluon 모델 훈련 완료!

📋 Category2 모델 리더보드:
                    model  score_val eval_metric  pred_time_val    fit_time  \
0     WeightedEnsemble_L3   0.667043    f1_macro       0.696630  130.190227   
1     WeightedEnsemble_L2   0.653207    f1_macro       0.448575  109.940131   
2  NeuralNetFastAI_BAG_L2   0.645488    f1_macro       0.639618  123.253865   
3  NeuralNetFastAI_BAG_L1   0.636916    f1_macro       0.168512   12.763404   
4         LightGBM_BAG_L2   0.593968    f1_macro       0.498585  116.274353   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.004001           0.339079            3       True   
1                0.003000           0.263061            2       True   
2                0.194044          13.576795            2       True   
3                0.168512          12.763404            1       True   
4                0.053010           6.597283            2       True   

   fit_order  
0          8  
1       

In [ ]:
# 15. AutoGluon 복합 라벨 방식 - 두 카테고리 조합을 하나의 라벨로 예측

print("🔗 AutoGluon 복합 라벨 방식 - 카테고리 조합 예측")
print("="*80)

try:
    from autogluon.tabular import TabularPredictor
    print("✅ AutoGluon 라이브러리 로드 성공")
    autogluon_available = True
except ImportError:
    print("❌ AutoGluon이 설치되지 않았습니다. 다음 명령으로 설치하세요:")
    print("pip install autogluon")
    autogluon_available = False
    
import os
import warnings
warnings.filterwarnings('ignore')

# 1. 복합 라벨 데이터 준비
print("\n📊 복합 라벨 데이터 준비...")

# 훈련 데이터: 벡터 + 복합 라벨
train_combined_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
train_combined_df['category1'] = y
train_combined_df['category2'] = data['re_category2'].values

# 두 카테고리를 결합한 복합 라벨 생성 (예: "기쁨_감동", "슬픔_절망")
train_combined_df['combined_label'] = train_combined_df['category1'] + "_" + train_combined_df['category2']

# 테스트 데이터: 벡터 + 실제 라벨  
test_combined_df = pd.DataFrame(test_X, columns=[f'feature_{i}' for i in range(test_X.shape[1])])
test_combined_df['category1'] = test_y_actual
test_combined_df['category2'] = test_y_actual_cat2
test_combined_df['combined_label'] = test_combined_df['category1'] + "_" + test_combined_df['category2']

print(f"훈련 데이터 크기: {train_combined_df.shape}")
print(f"테스트 데이터 크기: {test_combined_df.shape}")
print(f"복합 라벨 종류: {len(train_combined_df['combined_label'].unique())}개")

# 복합 라벨 분포 확인
print(f"\n📈 복합 라벨 상위 15개 분포:")
label_counts = train_combined_df['combined_label'].value_counts()
print(label_counts.head(15))

# 희소한 라벨 확인
rare_labels = label_counts[label_counts < 3]  # 3개 미만인 라벨
if len(rare_labels) > 0:
    print(f"\n⚠️ 희소한 라벨 ({len(rare_labels)}개): {rare_labels.index.tolist()}")
    print("이러한 라벨들은 훈련이 어려울 수 있습니다.")

# 2. 복합 라벨 AutoGluon 모델 훈련
combined_save_path = "autogluon_combined_label_model"
if os.path.exists(combined_save_path):
    import shutil
    shutil.rmtree(combined_save_path)
    print(f"기존 복합 라벨 모델 폴더 {combined_save_path} 삭제됨")

print(f"\n🤖 복합 라벨 AutoGluon 모델 훈련 시작...")
print("="*60)

combined_predictor = None
if autogluon_available:
    try:
        # 복합 라벨 예측기 생성
        combined_predictor = TabularPredictor(
            label='combined_label', 
            path=combined_save_path,
            eval_metric='f1_macro',  # F1 점수 최적화
            problem_type='multiclass'
        )
        
        # 모델 훈련 (여러 알고리즘 자동 시도)
        print("복합 라벨 모델들 훈련 중... (예상 소요 시간: 5-10분)")
        combined_predictor = combined_predictor.fit(
            train_data=train_combined_df.drop(['category1', 'category2'], axis=1),
            time_limit=300,  # 5분 제한
            presets='medium_quality_faster_train',
            num_bag_folds=3,  # 앙상블을 위한 폴드 수
            num_bag_sets=1,
            num_stack_levels=1
        )
        
        print("✅ AutoGluon 복합 라벨 모델 훈련 완료!")
        
        # 리더보드 출력
        print(f"\n📋 복합 라벨 AutoGluon 모델 리더보드:")
        combined_leaderboard = combined_predictor.leaderboard()
        print(combined_leaderboard.head())
        
        # 최고 성능 모델 정보
        best_combined_model = combined_leaderboard.iloc[0]['model']
        best_combined_score = combined_leaderboard.iloc[0]['score_val']
        print(f"\n🏆 최고 성능 복합 모델: {best_combined_model}")
        print(f"검증 F1-Score: {best_combined_score:.4f}")
        
    except Exception as e:
        print(f"❌ AutoGluon 복합 라벨 훈련 실패: {str(e)}")
        combined_predictor = None

# 3. XGBoost 대안 모델 (AutoGluon 실패시)
if not autogluon_available or combined_predictor is None:
    print(f"\n🔄 XGBoost 복합 라벨 모델로 진행...")
    from sklearn.preprocessing import LabelEncoder
    
    # 복합 라벨 인코딩
    combined_le = LabelEncoder()
    y_combined_encoded = combined_le.fit_transform(train_combined_df['combined_label'])
    
    print(f"복합 라벨 인코딩: {len(combined_le.classes_)}개 클래스")
    
    # XGBoost 모델 훈련 (복합 라벨용 고성능 설정)
    combined_xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.03,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    
    print("XGBoost 복합 라벨 모델 훈련 중...")
    combined_xgb_model.fit(X, y_combined_encoded)
    
    print("✅ XGBoost 복합 라벨 모델 훈련 완료!")

# 4. 복합 라벨 모델로 예측 수행
print(f"\n🎯 복합 라벨 모델 예측 수행...")
print("="*60)

if autogluon_available and combined_predictor is not None:
    print("AutoGluon 복합 라벨 모델로 예측...")
    
    # 테스트 데이터 예측
    test_pred_combined_labels = combined_predictor.predict(test_combined_df.drop(['category1', 'category2', 'combined_label'], axis=1))
    test_pred_combined_proba = combined_predictor.predict_proba(test_combined_df.drop(['category1', 'category2', 'combined_label'], axis=1))
    
    model_type = "AutoGluon 복합"
    
else:
    print("XGBoost 복합 라벨 모델로 예측...")
    
    # 예측
    test_pred_combined_encoded = combined_xgb_model.predict(test_X)
    test_pred_combined_labels = combined_le.inverse_transform(test_pred_combined_encoded)
    
    model_type = "XGBoost 복합"

# 5. 복합 라벨을 개별 카테고리로 분리
print(f"복합 라벨을 개별 카테고리로 분리...")

# 예측된 복합 라벨을 개별 카테고리로 분리
if isinstance(test_pred_combined_labels, pd.Series):
    test_pred_split = test_pred_combined_labels.str.split('_', expand=True)
elif isinstance(test_pred_combined_labels, np.ndarray):
    test_pred_split = pd.Series(test_pred_combined_labels).str.split('_', expand=True)

test_pred_cat1_combined = test_pred_split[0].values
test_pred_cat2_combined = test_pred_split[1].values

print(f"✅ {model_type} 복합 라벨 예측 완료!")

# 6. 성능 평가
print(f"\n📊 {model_type} 모델 성능 평가")
print("="*60)

# 개별 카테고리 정확도
cat1_accuracy_combined = (test_pred_cat1_combined == test_y_actual).mean()
cat2_accuracy_combined = (test_pred_cat2_combined == test_y_actual_cat2).mean()

print(f"Category1 정확도: {cat1_accuracy_combined:.4f} ({cat1_accuracy_combined*100:.2f}%)")
print(f"Category2 정확도: {cat2_accuracy_combined:.4f} ({cat2_accuracy_combined*100:.2f}%)")

# 🎯 핵심 지표: 두 카테고리 모두 정답 (복합 라벨의 핵심 장점)
both_correct_combined = (test_pred_cat1_combined == test_y_actual) & (test_pred_cat2_combined == test_y_actual_cat2)
both_accuracy_combined = both_correct_combined.mean()
both_count_combined = both_correct_combined.sum()

print(f"\n🎯 핵심 지표 - 두 카테고리 모두 정답:")
print(f"정확도: {both_accuracy_combined:.4f} ({both_accuracy_combined*100:.2f}%)")
print(f"정답 개수: {both_count_combined}/{len(test_y_actual)}")

# 복합 라벨 직접 비교 (가장 정확한 지표)
test_actual_combined_labels = test_combined_df['combined_label'].values
combined_direct_accuracy = (test_pred_combined_labels == test_actual_combined_labels).mean()
print(f"복합 라벨 직접 정확도: {combined_direct_accuracy:.4f} ({combined_direct_accuracy*100:.2f}%)")

# F1-Score 계산 (복합 라벨 기준)
from sklearn.metrics import f1_score, classification_report
f1_combined_direct = f1_score(test_actual_combined_labels, test_pred_combined_labels, average='macro')
print(f"복합 라벨 F1-Score (macro): {f1_combined_direct:.4f}")

# 7. 모든 방식 성능 비교
print(f"\n📈 모델 방식 종합 비교:")
print("-"*80)
print(f"{'방식':<20} {'Cat1 정확도':<12} {'Cat2 정확도':<12} {'종합 정확도':<12} {'개선':<10}")
print("-"*80)

# 기존 개별 모델 방식
original_both_accuracy = (test_y_pred == test_y_actual) & (test_y_pred_cat2 == test_y_actual_cat2)
original_both_accuracy = original_both_accuracy.mean()
print(f"{'기존 개별 모델':<20} {accuracy:<12.4f} {accuracy_cat2:<12.4f} {original_both_accuracy:<12.4f} {'기준':<10}")

# 복합 라벨 방식
improvement_combined = both_accuracy_combined - original_both_accuracy
improvement_text_combined = f"+{improvement_combined:.4f}" if improvement_combined > 0 else f"{improvement_combined:.4f}"
print(f"{model_type:<20} {cat1_accuracy_combined:<12.4f} {cat2_accuracy_combined:<12.4f} {both_accuracy_combined:<12.4f} {improvement_text_combined:<10}")

# 8. 복합 라벨 상세 분류 리포트 (상위 클래스)
print(f"\n📋 복합 라벨 Classification Report (상위 20개 클래스):")
print("-"*80)

# 가장 빈번한 복합 라벨들만 보고서 생성
top_combined_labels = pd.Series(test_actual_combined_labels).value_counts().head(20).index.tolist()
mask_combined = pd.Series(test_actual_combined_labels).isin(top_combined_labels)

if mask_combined.sum() > 0:
    filtered_actual_combined = pd.Series(test_actual_combined_labels)[mask_combined]
    filtered_pred_combined = pd.Series(test_pred_combined_labels)[mask_combined]
    
    combined_report = classification_report(
        filtered_actual_combined, 
        filtered_pred_combined, 
        target_names=top_combined_labels, 
        labels=top_combined_labels,
        zero_division=0
    )
    print(combined_report)

# 9. 복합 라벨 예측 샘플 분석
print(f"\n🔍 복합 라벨 예측 샘플 분석 (처음 15개):")
print("="*100)

for i in range(min(15, len(test_y_actual))):
    text = test_texts[i][:50] + "..." if len(test_texts[i]) > 50 else test_texts[i]
    
    actual_combined = test_actual_combined_labels[i]
    pred_combined = test_pred_combined_labels[i]
    
    # 개별 카테고리 분리
    actual_cat1 = test_y_actual[i]
    actual_cat2 = test_y_actual_cat2[i]
    pred_cat1 = test_pred_cat1_combined[i]
    pred_cat2 = test_pred_cat2_combined[i]
    
    # 매칭 상태
    combined_match = actual_combined == pred_combined
    both_match = (actual_cat1 == pred_cat1) and (actual_cat2 == pred_cat2)
    
    status = "✅" if combined_match else "❌"
    
    print(f"{i+1:2d}. {status} 복합: {actual_combined} → {pred_combined}")
    print(f"      개별: {actual_cat1}|{actual_cat2} → {pred_cat1}|{pred_cat2}")
    print(f"      텍스트: {text}")
    print()

# 10. 복합 라벨 방식의 장단점 분석
print(f"\n🔬 복합 라벨 방식 분석:")
print("="*60)

print(f"📈 장점:")
print(f"1. 카테고리 간 의존성을 직접적으로 학습")
print(f"2. 한 번의 예측으로 두 카테고리 동시 결정")
print(f"3. 카테고리 조합의 빈도를 고려한 학습")

print(f"\n📉 단점:")
print(f"1. 희소한 조합에 대한 학습 어려움")
print(f"2. 새로운 조합 출현 시 대응 곤란")
print(f"3. 클래스 수 증가로 인한 복잡도 상승")

print(f"\n💡 복합 라벨 방식 결론:")
if improvement_combined > 0.01:
    print(f"✅ 복합 라벨 방식이 {improvement_combined*100:.2f}%p 향상! 카테고리 의존성이 강한 경우 효과적")
elif improvement_combined > -0.01:
    print(f"📊 개별 모델과 유사한 성능. 데이터 특성에 따라 선택")
else:
    print(f"⚠️ 개별 모델이 {abs(improvement_combined)*100:.2f}%p 더 좋음. 희소한 조합 때문일 가능성")

if autogluon_available and combined_predictor is not None:
    print(f"\n💾 복합 라벨 모델 저장 위치: {combined_save_path}")
    print(f"📥 모델 로드 방법:")
    print(f"combined_predictor = TabularPredictor.load('{combined_save_path}')")
    print(f"combined_pred = combined_predictor.predict(new_data)")
    print(f"# 복합 라벨을 개별 카테고리로 분리")
    print(f"cat1_pred, cat2_pred = combined_pred.str.split('_', expand=True)[0], combined_pred.str.split('_', expand=True)[1]")

print(f"\n🎯 복합 라벨 최종 성과:")
print(f"두 카테고리 동시 정답률: {both_accuracy_combined*100:.2f}% ({both_count_combined}/{len(test_y_actual)}개)")
print(f"복합 라벨 직접 정확도: {combined_direct_accuracy*100:.2f}%")

Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       14.01 GB / 31.91 GB (43.9%)
Disk Space Avail:   87.85 GB / 465.12 GB (18.9%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'


🔗 AutoGluon 복합 라벨 방식 - 카테고리 조합 예측
✅ AutoGluon 라이브러리 로드 성공

📊 복합 라벨 데이터 준비...
훈련 데이터 크기: (3359, 1027)
테스트 데이터 크기: (664, 1027)
복합 라벨 종류: 77개

📈 복합 라벨 상위 15개 분포:
combined_label
슬픔_무기력      98
수치심_부끄러움    96
기쁨_편안함      92
슬픔_외로움      90
두려움_놀람      84
슬픔_허망       78
기쁨_안정감      78
기쁨_공감       77
기쁨_기대감      77
분노_불쾌       72
사랑_두근거림     71
슬픔_그리움      67
기쁨_감동       67
사랑_다정함      65
기쁨_고마움      64
Name: count, dtype: int64

⚠️ 희소한 라벨 (8개): ['중립_공감', '수치심_수치심', '기쁨_귀중함', '미움(상대방)_불쾌', '슬픔_죄책감', '슬픔_답답함', '사랑_공감', '슬픔_공감']
이러한 라벨들은 훈련이 어려울 수 있습니다.

🤖 복합 라벨 AutoGluon 모델 훈련 시작...
복합 라벨 모델들 훈련 중... (예상 소요 시간: 5-10분)


Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "c:\Users\user\Desktop\SKN_AFTER_STUDY\autogluon_combined_label_model"
Train Data Rows:    3359
Train Data Columns: 1024
Label Column:       combined_label
Problem Type:       multiclass
Preprocessing data ...
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.993450431676094
Train Data Class Count: 67
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    14305.22 MB
	Train Data (Original)  Memory Usage: 26.07 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUnique

✅ AutoGluon 복합 라벨 모델 훈련 완료!

📋 복합 라벨 AutoGluon 모델 리더보드:
                    model  score_val eval_metric  pred_time_val    fit_time  \
0     WeightedEnsemble_L3   0.622779    f1_macro       0.972064  277.438740   
1  NeuralNetFastAI_BAG_L2   0.611854    f1_macro       0.776809  199.168023   
2     WeightedEnsemble_L2   0.592764    f1_macro       0.559211  185.839937   
3  NeuralNetFastAI_BAG_L1   0.592117    f1_macro       0.197045   14.014211   
4         LightGBM_BAG_L2   0.539056    f1_macro       0.606224  196.524052   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.003000           0.368226            3       True   
1                0.220598          13.610406            2       True   
2                0.003000           0.282319            2       True   
3                0.197045          14.014211            1       True   
4                0.050013          10.966435            2       True   

   fit_order  
0          8  
1     

: 

In [12]:
# 복합 라벨 방식으로 로지스틱 회귀와 SVM Linear 모델 비교
print("=" * 80)
print("🔬 복합 라벨 방식: 로지스틱 회귀 vs SVM Linear 비교")
print("=" * 80)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np
import pandas as pd

# 1. 데이터 준비 (기존 임베딩 벡터 사용)
print("📊 데이터 준비 중...")

# 훈련 데이터 준비 (data 변수의 임베딩 벡터 사용)
X_train = np.vstack(data['vector'].values)  # 1024차원 임베딩 벡터
train_cat1 = data['re_category1'].values
train_cat2 = data['re_category2'].values

# 테스트 데이터 준비 (test_data의 임베딩 벡터 생성)
print("📝 테스트 데이터 벡터 생성 중...")
test_texts = test_data['context'].fillna('').astype(str).tolist()
test_vectors = [embeddings_model.encode(text).tolist() for text in test_texts]
X_test = np.vstack(test_vectors)  # 1024차원 임베딩 벡터

test_cat1_actual = test_data['category1'].values
test_cat2_actual = test_data['category2'].values

# 복합 라벨 생성 (category1_category2 형태)
train_combined_labels = [f"{cat1}_{cat2}" for cat1, cat2 in zip(train_cat1, train_cat2)]
test_combined_labels = [f"{cat1}_{cat2}" for cat1, cat2 in zip(test_cat1_actual, test_cat2_actual)]

print(f"훈련 데이터: {len(X_train)}개")
print(f"테스트 데이터: {len(X_test)}개")
print(f"벡터 차원: {X_train.shape[1]}차원")
print(f"복합 라벨 종류: {len(set(train_combined_labels))}개")

# 훈련/테스트 라벨 분석
train_label_set = set(train_combined_labels)
test_label_set = set(test_combined_labels)
unseen_labels = test_label_set - train_label_set
common_labels = train_label_set & test_label_set

print(f"훈련 데이터 고유 라벨: {len(train_label_set)}개")
print(f"테스트 데이터 고유 라벨: {len(test_label_set)}개")
print(f"공통 라벨: {len(common_labels)}개")
print(f"테스트에만 있는 라벨: {len(unseen_labels)}개")

if unseen_labels:
    print(f"처리할 수 없는 라벨들: {sorted(list(unseen_labels))[:5]}...")
    
    # 공통 라벨만 사용하여 필터링
    train_mask = np.array([label in common_labels for label in train_combined_labels])
    test_mask = np.array([label in common_labels for label in test_combined_labels])
    
    X_train_filtered = X_train[train_mask]
    train_combined_filtered = [label for i, label in enumerate(train_combined_labels) if train_mask[i]]
    
    X_test_filtered = X_test[test_mask]
    test_combined_filtered = [label for i, label in enumerate(test_combined_labels) if test_mask[i]]
    test_cat1_filtered = test_cat1_actual[test_mask]
    test_cat2_filtered = test_cat2_actual[test_mask]
    
    print(f"필터링 후 훈련 데이터: {len(X_train_filtered)}개")
    print(f"필터링 후 테스트 데이터: {len(X_test_filtered)}개")
else:
    X_train_filtered = X_train
    train_combined_filtered = train_combined_labels
    X_test_filtered = X_test
    test_combined_filtered = test_combined_labels
    test_cat1_filtered = test_cat1_actual
    test_cat2_filtered = test_cat2_actual

# 2. 라벨 인코딩
le_combined = LabelEncoder()
y_train_encoded = le_combined.fit_transform(train_combined_filtered)
y_test_encoded = le_combined.transform(test_combined_filtered)

print(f"인코딩된 라벨 종류: {len(le_combined.classes_)}")

# 3. 로지스틱 회귀 모델 훈련
print("\n🤖 로지스틱 회귀 모델 훈련 중...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, multi_class='ovr')
lr_model.fit(X_train_filtered, y_train_encoded)

# 로지스틱 회귀 예측
lr_pred_encoded = lr_model.predict(X_test_filtered)
lr_pred_labels = le_combined.inverse_transform(lr_pred_encoded)

# 로지스틱 회귀 결과 분석
lr_accuracy = accuracy_score(test_combined_filtered, lr_pred_labels)
lr_f1_macro = f1_score(y_test_encoded, lr_pred_encoded, average='macro')
lr_f1_weighted = f1_score(y_test_encoded, lr_pred_encoded, average='weighted')

# 개별 카테고리 정확도 계산 (로지스틱 회귀)
lr_cat1_pred = [label.split('_')[0] for label in lr_pred_labels]
lr_cat2_pred = [label.split('_')[1] for label in lr_pred_labels]
lr_cat1_accuracy = accuracy_score(test_cat1_filtered, lr_cat1_pred)
lr_cat2_accuracy = accuracy_score(test_cat2_filtered, lr_cat2_pred)
lr_both_correct = sum(1 for i in range(len(test_cat1_filtered)) 
                     if lr_cat1_pred[i] == test_cat1_filtered[i] and lr_cat2_pred[i] == test_cat2_filtered[i])
lr_both_accuracy = lr_both_correct / len(test_cat1_filtered)

# 4. SVM Linear 모델 훈련
print("🤖 SVM Linear 모델 훈련 중...")
svm_model = SVC(kernel='linear', random_state=42, probability=True)
svm_model.fit(X_train_filtered, y_train_encoded)

# SVM 예측
svm_pred_encoded = svm_model.predict(X_test_filtered)
svm_pred_labels = le_combined.inverse_transform(svm_pred_encoded)

# SVM 결과 분석
svm_accuracy = accuracy_score(test_combined_filtered, svm_pred_labels)
svm_f1_macro = f1_score(y_test_encoded, svm_pred_encoded, average='macro')
svm_f1_weighted = f1_score(y_test_encoded, svm_pred_encoded, average='weighted')

# 개별 카테고리 정확도 계산 (SVM)
svm_cat1_pred = [label.split('_')[0] for label in svm_pred_labels]
svm_cat2_pred = [label.split('_')[1] for label in svm_pred_labels]
svm_cat1_accuracy = accuracy_score(test_cat1_filtered, svm_cat1_pred)
svm_cat2_accuracy = accuracy_score(test_cat2_filtered, svm_cat2_pred)
svm_both_correct = sum(1 for i in range(len(test_cat1_filtered)) 
                      if svm_cat1_pred[i] == test_cat1_filtered[i] and svm_cat2_pred[i] == test_cat2_filtered[i])
svm_both_accuracy = svm_both_correct / len(test_cat1_filtered)

# 5. 결과 출력
print("\n" + "="*80)
print("📊 모델 성능 비교 결과")
print("="*80)

print(f"\n🔸 로지스틱 회귀 결과:")
print(f"  - 복합 라벨 정확도: {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")
print(f"  - Category1 정확도: {lr_cat1_accuracy:.4f} ({lr_cat1_accuracy*100:.2f}%)")
print(f"  - Category2 정확도: {lr_cat2_accuracy:.4f} ({lr_cat2_accuracy*100:.2f}%)")
print(f"  - 두 카테고리 모두 정답: {lr_both_correct}/{len(test_cat1_filtered)} ({lr_both_accuracy*100:.2f}%)")
print(f"  - F1-Score (Macro): {lr_f1_macro:.4f}")
print(f"  - F1-Score (Weighted): {lr_f1_weighted:.4f}")

print(f"\n🔸 SVM Linear 결과:")
print(f"  - 복합 라벨 정확도: {svm_accuracy:.4f} ({svm_accuracy*100:.2f}%)")
print(f"  - Category1 정확도: {svm_cat1_accuracy:.4f} ({svm_cat1_accuracy*100:.2f}%)")
print(f"  - Category2 정확도: {svm_cat2_accuracy:.4f} ({svm_cat2_accuracy*100:.2f}%)")
print(f"  - 두 카테고리 모두 정답: {svm_both_correct}/{len(test_cat1_filtered)} ({svm_both_accuracy*100:.2f}%)")
print(f"  - F1-Score (Macro): {svm_f1_macro:.4f}")
print(f"  - F1-Score (Weighted): {svm_f1_weighted:.4f}")

# 성능 비교
print(f"\n🏆 성능 비교:")
better_complex = "로지스틱 회귀" if lr_accuracy > svm_accuracy else "SVM Linear"
better_both = "로지스틱 회귀" if lr_both_accuracy > svm_both_accuracy else "SVM Linear"
better_f1 = "로지스틱 회귀" if lr_f1_macro > svm_f1_macro else "SVM Linear"

print(f"  - 복합 라벨 정확도가 더 높은 모델: {better_complex}")
print(f"  - 두 카테고리 동시 정답률이 더 높은 모델: {better_both}")
print(f"  - F1-Score (Macro)가 더 높은 모델: {better_f1}")

# 차이 분석
acc_diff = abs(lr_accuracy - svm_accuracy)
both_diff = abs(lr_both_accuracy - svm_both_accuracy)
f1_diff = abs(lr_f1_macro - svm_f1_macro)

print(f"\n📈 성능 차이:")
print(f"  - 복합 라벨 정확도 차이: {acc_diff:.4f} ({acc_diff*100:.2f}%p)")
print(f"  - 두 카테고리 동시 정답률 차이: {both_diff:.4f} ({both_diff*100:.2f}%p)")
print(f"  - F1-Score 차이: {f1_diff:.4f}")

# 샘플 예측 결과 비교
print(f"\n📝 샘플 예측 결과 비교 (처음 10개):")
print("-" * 120)
print(f"{'No.':<3} {'실제 Cat1':<12} {'실제 Cat2':<12} {'LR Cat1':<12} {'LR Cat2':<12} {'SVM Cat1':<12} {'SVM Cat2':<12} {'LR 정답':<8} {'SVM 정답':<8}")
print("-" * 120)

for i in range(min(10, len(test_cat1_filtered))):
    lr_correct = "✓" if lr_cat1_pred[i] == test_cat1_filtered[i] and lr_cat2_pred[i] == test_cat2_filtered[i] else "✗"
    svm_correct = "✓" if svm_cat1_pred[i] == test_cat1_filtered[i] and svm_cat2_pred[i] == test_cat2_filtered[i] else "✗"
    
    print(f"{i+1:<3} {test_cat1_filtered[i]:<12} {test_cat2_filtered[i]:<12} "
          f"{lr_cat1_pred[i]:<12} {lr_cat2_pred[i]:<12} {svm_cat1_pred[i]:<12} {svm_cat2_pred[i]:<12} "
          f"{lr_correct:<8} {svm_correct:<8}")

print("="*80)

🔬 복합 라벨 방식: 로지스틱 회귀 vs SVM Linear 비교
📊 데이터 준비 중...
📝 테스트 데이터 벡터 생성 중...
훈련 데이터: 3359개
테스트 데이터: 664개
벡터 차원: 1024차원
복합 라벨 종류: 77개
훈련 데이터 고유 라벨: 77개
테스트 데이터 고유 라벨: 70개
공통 라벨: 68개
테스트에만 있는 라벨: 2개
처리할 수 없는 라벨들: ['중립_놀람', '중립_만족감']...
필터링 후 훈련 데이터: 3335개
필터링 후 테스트 데이터: 660개
인코딩된 라벨 종류: 68

🤖 로지스틱 회귀 모델 훈련 중...


c:\Users\user\anaconda3\envs\skn_after_study\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


🤖 SVM Linear 모델 훈련 중...

📊 모델 성능 비교 결과

🔸 로지스틱 회귀 결과:
  - 복합 라벨 정확도: 0.2939 (29.39%)
  - Category1 정확도: 0.5030 (50.30%)
  - Category2 정확도: 0.3106 (31.06%)
  - 두 카테고리 모두 정답: 194/660 (29.39%)
  - F1-Score (Macro): 0.1735
  - F1-Score (Weighted): 0.2628

🔸 SVM Linear 결과:
  - 복합 라벨 정확도: 0.3197 (31.97%)
  - Category1 정확도: 0.5303 (53.03%)
  - Category2 정확도: 0.3394 (33.94%)
  - 두 카테고리 모두 정답: 211/660 (31.97%)
  - F1-Score (Macro): 0.1995
  - F1-Score (Weighted): 0.2982

🏆 성능 비교:
  - 복합 라벨 정확도가 더 높은 모델: SVM Linear
  - 두 카테고리 동시 정답률이 더 높은 모델: SVM Linear
  - F1-Score (Macro)가 더 높은 모델: SVM Linear

📈 성능 차이:
  - 복합 라벨 정확도 차이: 0.0258 (2.58%p)
  - 두 카테고리 동시 정답률 차이: 0.0258 (2.58%p)
  - F1-Score 차이: 0.0259

📝 샘플 예측 결과 비교 (처음 10개):
------------------------------------------------------------------------------------------------------------------------
No. 실제 Cat1      실제 Cat2      LR Cat1      LR Cat2      SVM Cat1     SVM Cat2     LR 정답    SVM 정답  
--------------------------------------------------------